In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as mcolors


make sure to rename the columns by removing é


In [ ]:
###### UNCOMMENT THIS IF YOU ARE WORKING ON COLAB #######


# from google.colab import drive
# import os

# # 1. This will ask for permission to access your Drive
# drive.mount('/content/drive')

# # 2. Navigate to your project folder
# # Check your Drive path - it usually starts with /content/drive/MyDrive/
# # Adjust the path below to match where you uploaded the folder
# path = "/content/drive/MyDrive/Data-Mining/data/raw"
# os.chdir(path)

# # 3. List files to make sure you are in the right spot
# print("Successfully connected! Files in folder:")
# print(os.listdir())

# Exploratory Data Analysis (EDA) for Packages and Receptacle Datasets

In [ ]:
###### UNCOMMENT THIS IF YOU ARE WORKING ON COLAB #######

# packages_df = pd.read_csv('packages_data_2023_2025.csv',delimiter=';', encoding='latin-1')
# receptacles_df = pd.read_csv('receptacle_data_2023_2025.csv',delimiter=';', encoding='latin-1')

In [ ]:
# Load datasets
packages_df = pd.read_csv('../data/raw/packages_data_2023_2025.csv',delimiter=';', encoding='latin-1')
receptacles_df = pd.read_csv('../data/raw/receptacle_data_2023_2025.csv',delimiter=';', encoding='latin-1')

* Columns' names adjustment

In [ ]:
packages_df = packages_df.rename(columns={'établissement_postal': 'etablissement_postal', 'next_établissement_postal': 'next_etablissement_postal'})
receptacles_df = receptacles_df.rename(columns={'ï»¿RECPTCL_FID': 'RECPTCL_FID', 'EVENT_TYPECD': 'EVENT_TYPE_CD', 'nextetablissement_postal': 'next_etablissement_postal'})

In [ ]:
packages_df.shape, receptacles_df.shape

* Columns' types adjustment

In [ ]:
packages_df['date'] = pd.to_datetime(packages_df['date'])
receptacles_df['date'] = pd.to_datetime(receptacles_df['date'])
packages_df['RECPTCL_FID'] = packages_df['RECPTCL_FID'].str.strip()
packages_df['MAILITM_FID'] = packages_df['MAILITM_FID'].str.strip()
packages_df['etablissement_postal'] = packages_df['etablissement_postal'].str.strip()
packages_df['next_etablissement_postal'] = packages_df['next_etablissement_postal'].str.strip()
receptacles_df['etablissement_postal'] = receptacles_df['etablissement_postal'].str.strip()
receptacles_df['next_etablissement_postal'] = receptacles_df['next_etablissement_postal'].str.strip()
receptacles_df['RECPTCL_FID'] = receptacles_df['RECPTCL_FID'].str.strip()

In [ ]:
raw_clustering_pkg = packages_df.copy()
raw_clustering_rcp = receptacles_df.copy()

In [ ]:
packages_df.info()


In [ ]:
receptacles_df.info()


In [ ]:
packages_df.head(n=5)

In [ ]:
receptacles_df.head(n=5)

 Initial Observations
- both datasets cover the period from 2023 to 2025
- we have no target variable in either dataset
- for packages dataset:
    - 6 features in total with 5 categorical and 1 numerical
    - MAILITM_FID is unique identifier for each package
    - RECPTCL_FID is foreign key linking to receptacle dataset
    - etablissement_postal and next_etablissement_postal have some null values
- for receptacle dataset:
    - 5 features in total with 4 categorical and 1 numerical
    - RECPTCL_FID is unique identifier for each receptacle
    - EVENT_TYPE_CD has some null values

In [ ]:
for column in packages_df.columns:
    print(f'{column} has {packages_df[column].nunique()} unique values.')
    print(f'{column} has {packages_df[column].isnull().sum()} null values.')
    print("\n")


In [ ]:
for column in receptacles_df.columns:
    print(f'{column} has {receptacles_df[column].nunique()} unique values.')
    print(f'{column} has {receptacles_df[column].isnull().sum()} null values.')
    print("\n")

we notice the following:<br>
- receptacle dataset has more unique values for RECPTCL_FID than packages dataset, indicating one-to-many relationship<br>
- MAILITM_FID is unique in packages dataset.<br>
- packages dataset have more unique date values than receptacle dataset.<br>
- both datasets have null values in etablissement_postal and next_etablissement_postal columns. This requires processing later on<br>
- packages dataset has more unique values in the next_etablissement_postal column compared to receptacle dataset but also more null values. **further investigation is needed to understand why**<br>


In [ ]:
packages_df.describe()

In [ ]:
receptacles_df.describe()

for EVENT_TYPE_CD we notice different range of values for packages and receptacle datasets indicating different types of events.<br>

#### Visualizations
for now we will visualize the distribution of EVENT_TYPE_CD in both datasets.<br>



In [ ]:
# Distribution of EVENT_TYPE_CD in packages dataset
plt.figure(figsize=(12, 6))
sns.countplot(data=packages_df,x='EVENT_TYPE_CD')
plt.title('distribution of EVENT_TYPE_CD in packages dataset')
plt.xlabel('EVENT_TYPE_CD')
plt.ylabel('count')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Distribution of EVENT_TYPE_CD in receptacle dataset
plt.figure(figsize=(12, 6))
sns.countplot(data=receptacles_df,x='EVENT_TYPE_CD')
plt.title('distribution of EVENT_TYPE_CD in Receptacle dataset')
plt.xlabel('EVENT_TYPE_CD')
plt.ylabel('count')
plt.xticks(rotation=45)
plt.show()

for etablissement_postal and next_etablissement_postal I will start with visualizing the receptacle dataset since the packages dataset has a lot of unique values<br>

In [ ]:
# Distribution of etablissement_postal in receptacle dataset
plt.figure(figsize=(12, 6))
sns.countplot(data=receptacles_df,x='etablissement_postal')
plt.title('distribution of etablissement_postal in Receptacle dataset')
plt.xlabel('etablissement_postal')
plt.ylabel('count')
plt.xticks(rotation=90)
plt.show()


In [ ]:
# Distribution of next_etablissement_postal in receptacle dataset
plt.figure(figsize=(12, 6))
sns.countplot(data=receptacles_df,x='next_etablissement_postal')
plt.title('distribution of next_etablissement_postal in Receptacle dataset')
plt.xlabel('next_etablissement_postal')
plt.ylabel('count')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# 1.  Top 10 locations to keep the map small
top_locs = receptacles_df['etablissement_postal'].value_counts().head(10).index
top_next = receptacles_df['next_etablissement_postal'].value_counts().head(10).index

# 2. include only these top locations
filtered_df = receptacles_df[
    (receptacles_df['etablissement_postal'].isin(top_locs)) &
    (receptacles_df['next_etablissement_postal'].isin(top_next))
]

# 3. Create a Matrix (Cross-tabulation)
matrix = pd.crosstab(filtered_df['etablissement_postal'], filtered_df['next_etablissement_postal'])

# 4. Plot Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(matrix, cmap='Reds', annot=True, fmt='d', linewidths=.5)
plt.title("Most Frequent Routes (Origin vs Destination) - Receptacle Dataset")
plt.xlabel("Next Destination")
plt.ylabel("Current Location")
plt.show()

we notice that some etablissements have significantly higher traffic compared to others, indicating  major distribution centers.<br>

for etablissement_postal and next_etablissement_postal we will create a heatmap to visualize the flow between current location and next destination.<br>
Count of parcels moving from A to B to see the density of connections between them

In [ ]:
# 1.  Top 10 locations to keep the map small
top_locs = packages_df['etablissement_postal'].value_counts().head(10).index
top_next = packages_df['next_etablissement_postal'].value_counts().head(10).index

# 2. include only these top locations
filtered_df = packages_df[
    (packages_df['etablissement_postal'].isin(top_locs)) &
    (packages_df['next_etablissement_postal'].isin(top_next))
]

# 3. Create a Matrix (Cross-tabulation)
matrix = pd.crosstab(filtered_df['etablissement_postal'], filtered_df['next_etablissement_postal'])

# 4. Plot Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(matrix, cmap='Reds', annot=True, fmt='d', linewidths=.5)
plt.title("Most Frequent Routes (Origin vs Destination) - Packages Dataset")
plt.xlabel("Next Destination")
plt.ylabel("Current Location")
plt.show()

In [ ]:
# Count packages per location
location_counts = packages_df['etablissement_postal'].value_counts().reset_index()
location_counts.columns = ['Location', 'Volume']

# keep only top 20 busiest centers
plt.figure(figsize=(12, 6))
sns.barplot(x='Volume', y='Location', data=location_counts.head(20), palette='viridis')
plt.title("Top 20 Busiest Postal Centers")
plt.xlabel("Number of Packages")
plt.ylabel("Center ID")
plt.show()

we notice the same pattern as before with some etablissements having significantly higher trafic compared to others.<br>

In [ ]:

# 1. Extract Time Features
packages_df['hour'] = packages_df['date'].dt.hour
packages_df['day_of_week'] = packages_df['date'].dt.day_name()

# 2. Create a Pivot Table (Cross-tabulation)
# Rows = Day, Cols = Hour, Values = Count of Scans
heatmap_data = pd.crosstab(
    packages_df['day_of_week'],
    packages_df['hour']
)

days_order = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
heatmap_data = heatmap_data.reindex(days_order)

# 3. Plot
plt.figure(figsize=(12, 6))
sns.heatmap(heatmap_data, cmap='YlOrRd', linewidths=.5, annot=False)
plt.title("Package Scan Activity by Day and Hour")
plt.xlabel("Hour of Day (0-23)")
plt.ylabel("Day of Week")
plt.show()

we notice that the busiest times for package scans are during weekdays, particularly from mid-morning to late afternoon.<br>

In [ ]:
#number of packages per receptacle
packages_per_receptacle = packages_df.groupby('RECPTCL_FID')['MAILITM_FID'].nunique()
packages_per_receptacle.head()


# Preprocessing

In [ ]:
packages_df.info()

In [ ]:
receptacles_df.info()

- Drop the packages starting from 2020 and keep only the ones starting from 2023

In [ ]:
#in packages df Drop the values that are from 2020 and start only from 2023
packages_df = packages_df[packages_df['date'].dt.year >= 2023]

In [ ]:
packages_df['etablissement_postal'].isna().sum()

* `etablissement_postal` have 26772 null values (2.7% of the whole dataset)
* As its null values are less than 5% of the dataset (2.7%), we drop these null values

In [ ]:
packages_df = packages_df[~packages_df['etablissement_postal'].isna()]
packages_df['etablissement_postal'].isna().sum()

* We propose to consider the packages having null `next_etablissement_postal`
as having issue during transfer, we'll try to validate that using
`EVENT_TYPE_CD` also
* Let's check if `EVENT_TYPE_CD` can indicate whether the `next_etablissement_postal` is null or not

In [ ]:
packages_unknown_next_etablissement = packages_df[packages_df['next_etablissement_postal'].isna()]
# keep only top EVENT_TYPES_ID
packages_unknown_next_etablissement = packages_unknown_next_etablissement['EVENT_TYPE_CD'].value_counts()

plt.figure(figsize=(12, 6))
packages_unknown_next_etablissement.head(10).plot(kind='bar')
plt.xlabel('EVENT TYPE CD')
plt.ylabel('Null Next Etablissement Postal')
plt.xticks(rotation=0)
plt.show()

* `EVENT_TYPE_CD` doesn't actually indicate null values of `next_etablissement_postal`

* the function `fill_NaN_next_etab` cell fills the `next_etablissement_postal` using the next `etablissement_postal` for the same package.
* if the last route for a specific package is null, then it keeps it null because there's no next `etablissement_postal` for that package.

In [ ]:
def fill_NaN_next_etab(df, id_col):
    # 1. Ensure the dataframe is sorted (same as before)
    df = df.sort_values([id_col, 'date'])

    # 2. Look ahead to the next row's postal code and ID
    shifted_postal = df['etablissement_postal'].shift(-1)
    shifted_id = df[id_col].shift(-1)
# 3. Identify the "boundaries" where the postal code changes within the same package
# This marks the last row of a block with the value of the start of the next block
    is_boundary = (df['etablissement_postal'] != shifted_postal) & \
              (df[id_col] == shifted_id)
# 4. Use grouped backfill to broadcast those values to all preceding rows in the block
# This replaces your 'blocks.map' logic with a single vectorized pass
    fill_values = shifted_postal.where(is_boundary).groupby(df[id_col]).bfill()

# 5. Fill only the NaNs in the existing column to match your original logic
    df['next_etablissement_postal'] = df['next_etablissement_postal'].fillna(fill_values)

    return df

In [ ]:
# Apply the function to fill NaN values in next_etablissement_postal
packages_df = fill_NaN_next_etab(packages_df, 'MAILITM_FID')
# Check remaining NaNs
packages_df.info()

* Like this, we've handled a good part of null values and inconsitencies for `packages` dataset.

* **We'll be doing the same steps for `receptacle` dataset**

In [ ]:
receptacles_df['etablissement_postal'].isna().sum()

* Dropping rows having null `etablissement_postal`, as they're just 0.1% of the data

In [ ]:
receptacles_df = receptacles_df[~receptacles_df['etablissement_postal'].isna()]
receptacles_df['etablissement_postal'].isna().sum()

* apply the function that fills null values of `next_etablissement_postal` using `etablissement_postal` to `receptacles_df`

In [ ]:
receptacles_df = fill_NaN_next_etab(receptacles_df, 'RECPTCL_FID')
# Check remaining NaNs
receptacles_df.info()

* Null values are mostly gone, but there are still some illogical packages' and receptacles' routes between `etablissements`
* We'll treat these logical routes now

In [ ]:
# for each package (group of rows), check whether there's any illogical route
# between 'etablissement_postal' and 'next_etablissement_postal'
def isPackageIllogical(group):
    return (
        group['next_etablissement_postal']
        .iloc[:-1]
        .ne(group['etablissement_postal'].shift(-1).iloc[:-1])
        .any()
    )

In [ ]:
illogical_packages = packages_df.groupby('MAILITM_FID').apply(isPackageIllogical)
illogical_packages.value_counts()

In [ ]:
103376 / packages_df['MAILITM_FID'].nunique()

* 103376 Packages have illogical routes (98%) of all packages, so it's impossible to drop them, but instead, we plan to ignore the `MAILITM_FID` and `RECPTCL_FID` in the training and testing sets that will come next

In [ ]:
# for each receptacle (group of rows), check whether there's any illogical route
# between 'etablissement_postal' and 'next_etablissement_postal'
def isReceptacleIllogical(group):
    return (
        group['next_etablissement_postal']
        .iloc[:-1]
        .ne(group['etablissement_postal'].shift(-1).iloc[:-1])
        .any()
    )

In [ ]:
illogical_receptacles = receptacles_df.groupby('RECPTCL_FID').apply(isReceptacleIllogical)
illogical_receptacles.value_counts()

In [ ]:
205519 / receptacles_df['RECPTCL_FID'].nunique()

205519 receptacles have illogical routes (95%) of all receptacles, so it's also impossible to drop them

In [ ]:
# keep copies for backup (en cas ou)
packages_df_copy = packages_df.copy()
receptacles_df_copy = receptacles_df.copy()

# Feature Extraction

### Check RECPTCL_FID and MAILITM_FID having same length formats
if yes then we can split them into meaningfull parts

In [ ]:
same=1
print("\n=== RECPTCL_FID  ===")
print(f"testing if the lengths of RECPTCL_FID values are all the same:")
for val in packages_df['RECPTCL_FID'].values:
    if len(str(val)) != 29 :
        print(f"  {val} (length: {len(str(val))})")
        same=0
        break
if same==1:
    print('all same length' )

same=1
print("\n=== MAILITM_FID  ===")
print(f"testing if the lengths of MAILITM_FID values are all the same:")
for val in packages_df['MAILITM_FID'].values:
    if len(str(val)) != 13 :
        print(f"  {val} (length: {len(str(val))})")
        same=0
        break
if same==1:
    print ('all same length' )



### RECPTCL_FID Analysis
- **Format:** 29-character string (e.g., `USORDADZALGDAUN30050001900005`)
- **Data Quality:** No null values (1,000,000) | 215,867 unique values in receptacle dataset and 45306 unique values in packages dataset
- **Extractable Features:**
  - Origin Country (2 chars): US, FR, AE, etc.
  - Destination Country (2 chars): DZ, AI, AA, etc.

### MAILITM_FID Analysis
- **Format:** 13-character string according to the S10-12 patern (e.g., `CA000132868US`, `CA000340856PK`)
- **Data Quality:** No null values (1,000,000 packages)
- **Extractable Features:**
  - Service Indicator (2 chars): CA, etc.
  - Serial Number (8 chars): 00013286, 00034085, etc.
  - Check Digit (1 char): 8, 6, etc.
  - Country Code (3 chars, right-stripped): US, PK.


## Definition of the parser funtions
These functions are responsible for spliting the IDs into parts

In [ ]:
def parse_recptcl_fid(id_str):
    origin_country = id_str[0:2]
    destination_country = id_str[6:8]
    return origin_country, destination_country

def parse_mailitm_fid(id_str):
    service_indicator = id_str[0:2]
    serial_number = id_str[2:11]
    country_code = id_str[11:14].strip()
    return service_indicator, serial_number, country_code


### Apply parsing functions

In [ ]:
# parsed_receptacles_df = receptacles_df.copy()
# parsed_receptacles_df[['origin_country', 'destination_country']] = parsed_receptacles_df['RECPTCL_FID'].apply(lambda x: pd.Series(parse_recptcl_fid(x)))

# Faster method:
parsed_data = list(receptacles_df['RECPTCL_FID'].apply(parse_recptcl_fid))

# Assign to new columns by creating a temporary DataFrame
parsed_receptacles_df = receptacles_df.copy()
parsed_receptacles_df[['origin_country', 'destination_country']] = pd.DataFrame(
    parsed_data, index=receptacles_df.index
)

In [ ]:
# parsed_packages_df = packages_df.copy()
# parsed_packages_df[['service_indicator', 'serial_number', 'country_code']] = parsed_packages_df['MAILITM_FID'].apply(lambda x: pd.Series(parse_mailitm_fid(x)))
# parsed_packages_df[['origin_country','destination_country']] = parsed_packages_df['RECPTCL_FID'].apply(lambda x: pd.Series(parse_recptcl_fid(x)))

# Faster method:
parsed_packages_df = packages_df.copy()

# 1. Optimize MAILITM_FID parsing
mailitm_data = list(parsed_packages_df['MAILITM_FID'].apply(parse_mailitm_fid))
parsed_packages_df[['service_indicator', 'serial_number', 'country_code']] = pd.DataFrame(
    mailitm_data, index=parsed_packages_df.index
)

# 2. Optimize RECPTCL_FID parsing
recptcl_data = list(parsed_packages_df['RECPTCL_FID'].apply(parse_recptcl_fid))
parsed_packages_df[['origin_country', 'destination_country']] = pd.DataFrame(
    recptcl_data, index=parsed_packages_df.index
)

### show samples of new parsed data

In [ ]:

print("=== packages_df sample with new parsed columns ===")
parsed_packages_df.head()



In [ ]:
print("\n=== receptacles_df sample with new parsed columns ===")
parsed_receptacles_df.head()

# Analysis of the extrcted features


## 1. parsed_packages_df

In [ ]:

print("\n--- Unique Value Counts for parsed_packages_df ---")
print("\nFor receptacle FID parsing:")
print(f"Unique origin_country values: {parsed_packages_df['origin_country'].nunique()}")
print(f"Unique destination_country values: {parsed_packages_df['destination_country'].nunique()}")

print("="*50)
print("\nFor mail item FID parsing:")
print(f"Unique service_indicator values: {parsed_packages_df['service_indicator'].nunique()}")
print(f"Unique country_code values: {parsed_packages_df['country_code'].nunique()}")



## 2. parsed_receptacles_df

In [ ]:
print("\n--- Unique Value Counts for parsed_receptacles_df ---")
print(f"Unique origin_country values: {parsed_receptacles_df['origin_country'].nunique()}")
print(f"Unique destination_country values: {parsed_receptacles_df['destination_country'].nunique()}")


## List values of the new columns obtained from receptacle FID parsing for both parsed dataframes

### 1. for parsed_packages_df

In [ ]:
#listing the values
print("\n--- Values of origin_country ---")
print(parsed_packages_df['origin_country'].unique())

print("\n--- Values of destination_country ---")
print(parsed_packages_df['destination_country'].unique())


### 2. for parsed_receptacles_df

In [ ]:
#listing the values
print("\n--- Values of origin_country ---")
print(parsed_receptacles_df['origin_country'].unique())

print("\n--- Values of destination_country ---")
print(parsed_receptacles_df['destination_country'].unique())


### Do the intersection of origin_country of both dataframes

In [ ]:
#do the intersection of origin_country values in both parsed datasets
packages_origin_countries = set(parsed_packages_df['origin_country'].unique())
receptacle_origin_countries = set(parsed_receptacles_df['origin_country'].unique())
common_origin_countries = packages_origin_countries.intersection(receptacle_origin_countries)
print("number of common origin_country values in both parsed datasets:", len(common_origin_countries))
print(f"\nCommon origin_country values in both paesed datasets: ")
print(common_origin_countries)
# remaining ones
remaining_in_packages = packages_origin_countries - common_origin_countries
remaining_in_receptacle = receptacle_origin_countries - common_origin_countries
print(f"Remaining origin_country values only in parsed_packages_df:")
print(remaining_in_packages)
print(f"Remaining origin_country values only in parsed_receptacles_df:")
print(remaining_in_receptacle )

### list the values of both service indicators and country code

### 1. service indicators

In [ ]:
print("\n--- Values of service_indicator ---")
print(parsed_packages_df['service_indicator'].unique())



 we can see that there are values that don't follow the standards in the S10-12 format so we need to handle that correctly

In [ ]:
#transform country_code to uppercase for consistency
parsed_packages_df['service_indicator'] = parsed_packages_df['service_indicator'].str.upper()
print("values of service_indicator after transformation to uppercase:")
print(parsed_packages_df['service_indicator'].unique())
print("number of unique service indicators after transformation:", parsed_packages_df['service_indicator'].nunique())


### 2. country code

In [ ]:
print("\n--- Values of country codes ---")

print(parsed_packages_df['country_code'].unique())

we can see that many values for the country codes are numbers instead of ISO 3166-1 format these values should be replaced by the values of origin country gotten from the receptacle when doing the preprocessing

### replace them with the correct origin country code

In [ ]:
# 1. Vectorized string capitalization
parsed_packages_df['country_code'] = parsed_packages_df['country_code'].str.upper()

# 2. Vectorized comparison to find mismatches
mismatch_mask = parsed_packages_df['origin_country'] != parsed_packages_df['country_code']

# 3. Count the Trues
count = mismatch_mask.sum()

print(f"Number of rows where origin_country does not match country_code: {count}")

### replace them

In [ ]:
# Use .loc to find rows where they don't match, and update only the 'country_code' column
parsed_packages_df.loc[parsed_packages_df['origin_country'] != parsed_packages_df['country_code'], 'country_code'] = parsed_packages_df['origin_country']

In [ ]:
#print the unique values again
print("\n--- Values of country codes after correction ---")
print(parsed_packages_df['country_code'].unique())
print("number of unique country codes after correction:", parsed_packages_df['country_code'].nunique())

## visualization of Origin Country distribution according to number of packages

### 1. for the parsed_packages_df

In [ ]:
origin_country_counts = parsed_packages_df['origin_country'].value_counts().head(20)
plt.figure(figsize=(12, 7))
plt.barh(origin_country_counts.index, origin_country_counts.values, color='steelblue')
plt.xlabel('Count', fontsize=11)
plt.ylabel('Origin Country', fontsize=11)
plt.title('Top 20 Origin Countries by packages count', fontsize=12, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

### 2. for the parsed_receptacles_df

In [ ]:
origin_country_counts = parsed_receptacles_df['origin_country'].value_counts().head(20)
plt.figure(figsize=(12, 7))
plt.barh(origin_country_counts.index, origin_country_counts.values, color='steelblue')
plt.xlabel('Count', fontsize=11)
plt.ylabel('Origin Country', fontsize=11)
plt.title('Top 20 Origin Countries by receptacle count', fontsize=12, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

## Visualiation of the service indicator

In [ ]:
service_indicator_count = parsed_packages_df['service_indicator'].value_counts().head(20)
plt.figure(figsize=(12, 7))
plt.barh(service_indicator_count.index, service_indicator_count.values, color='mediumseagreen')
plt.xlabel('Count', fontsize=11)
plt.ylabel('Service Indicator', fontsize=11)
plt.title('Top 20 Service Indicators by packages count', fontsize=12, fontweight='bold')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# Origin–Destination Flow Analysis

This section investigates the flow of receptacles and packages from origin countries to destination. We examine:
- packages count by origin country
- Top origin countries delivering to each destination
- Visual representation

### 1. parsed_packages_df

In [ ]:
# packages count by origin country
origin_country_volume = parsed_packages_df['origin_country'].value_counts()
print("\n--- Packages count by Origin Country ---")
print(origin_country_volume.head(15))

# Create origin_country × destination matrix
flow_matrix = pd.crosstab(parsed_packages_df['origin_country'],
                           parsed_packages_df['destination_country'])

# Keep only top 10 origin countries and top 10 destination countries for readability
top_origins = parsed_packages_df['origin_country'].value_counts().head(10).index
top_arrivals = parsed_packages_df['destination_country'].value_counts().head(10).index

flow_matrix_top = flow_matrix.loc[top_origins, top_arrivals]

print("\n--- Origin Country × destination Country (Top 10 × Top 10) ---")
print(flow_matrix_top)


In [ ]:
# Visualize origin × destination country flow
plt.figure(figsize=(14, 8))
sns.heatmap(flow_matrix_top, cmap='YlOrRd', annot=True, fmt='d',
            cbar_kws={'label': 'packages Count'}, linewidths=0.5)
plt.title('packages Flow: Origin Country × destination country (Top 10 × Top 10)',
          fontsize=14, fontweight='bold')
plt.xlabel('Arrival Hub', fontsize=12)
plt.ylabel('Origin Country', fontsize=12)
plt.tight_layout()
plt.show()


### 2. parsed_receptacles_df

In [ ]:
# receptacle count by origin country
origin_country_receptacle_volume = parsed_receptacles_df['origin_country'].value_counts()
print("\n--- Receptacle count by Origin Country ---")
print(origin_country_receptacle_volume.head(15))

destination__receptacle_volume=parsed_receptacles_df['destination_country'].value_counts()
print("\n--- Receptacle count by destination ---")
print(destination__receptacle_volume.head(15))

# Create origin_country × destination matrix
receptacle_flow_matrix = pd.crosstab(parsed_receptacles_df['origin_country'],
                           parsed_receptacles_df['destination_country'])

# Keep only top 10 origin countries and top 10 destination countries for readability
receptacle_top_origins = parsed_receptacles_df['origin_country'].value_counts().head(10).index
receptacle_top_arrivals = parsed_receptacles_df['destination_country'].value_counts().head(10).index

receptacle_flow_matrix_top = receptacle_flow_matrix.loc[receptacle_top_origins, receptacle_top_arrivals]

print("\n--- Origin Country × destination Country (Top 10 × Top 10) ---")
print(receptacle_flow_matrix_top)


In [ ]:
# Visualize origin × destination country flow
plt.figure(figsize=(14, 8))
sns.heatmap(receptacle_flow_matrix_top, cmap='YlOrRd', annot=True, fmt='d',
            cbar_kws={'label': 'receptacles Count'}, linewidths=0.5)
plt.title('receptacles Flow: Origin Country × destination country (Top 10 × Top 10)',
          fontsize=14, fontweight='bold')
plt.xlabel('Arrival Hub', fontsize=12)
plt.ylabel('Origin Country', fontsize=12)
plt.tight_layout()
plt.show()


### create pairs (origin, destination) for more detailed analysis

In [ ]:
parsed_packages_df['origin_destination'] = parsed_packages_df['origin_country'] + '_' + parsed_packages_df['destination_country']
parsed_receptacles_df['origin_destination'] = parsed_receptacles_df['origin_country'] + '_' + parsed_receptacles_df['destination_country']


listing the obtained values

In [ ]:
print('(origin_destination) pairs obtained for ')
print("\nfor parsed_packages_df :")
print("\nNumber of unique values", parsed_packages_df['origin_destination'].nunique())
print("\nValues : ")
print(parsed_packages_df['origin_destination'].unique())
print("\nfor parsed_receptacles_df :")
print("\nNumber of unique values", parsed_receptacles_df['origin_destination'].nunique())
print("\nValues : ")
print(parsed_receptacles_df['origin_destination'].unique())


### visualization of obtained results

### 1. parsed_packages_df

In [ ]:
origin_dest_counts = parsed_packages_df['origin_destination'].value_counts()
top_origin_dest_counts= origin_dest_counts.head(15)
plt.figure(figsize=(14, 7))
top_origin_dest_counts.plot(kind='bar')
plt.title('Histogram of (origin, destination) Pairs Based on Package Counts')
plt.xlabel('Origin_Destination')
plt.ylabel('Number of Packages')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()



### 2. parsed_receptacles_df

In [ ]:
origin_dest_counts = parsed_receptacles_df['origin_destination'].value_counts()
top_origin_dest_counts = origin_dest_counts.head(15)

plt.figure(figsize=(14, 7))
top_origin_dest_counts.plot(kind='bar')
plt.title('Histogram of (origin, destination) Pairs Based on receptacle Counts')
plt.xlabel('Origin_Destination')
plt.ylabel('Number of receptacles')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()


## origin_destination X etablissments analysis

## 1. current etablissment

### a. parsed_packages_df

In [ ]:
# For parsed_packages_df: visualize the histogram of counts by (origin_destination, etablissement_postal) pairs

# First, select top N most common (origin_destination, etablissement_postal) pairs for clarity
pair_counts_current = parsed_packages_df.groupby(['origin_destination', 'etablissement_postal']).size().reset_index(name='count')

# Sort and keep only top N pairs (by count)
top_n = 20
top_pairs_current = pair_counts_current.sort_values('count', ascending=False).head(top_n)

plt.figure(figsize=(16, 8))
sns.barplot(
    data=top_pairs_current,
    x='count',
    y=top_pairs_current.apply(lambda x: f"{x['origin_destination']} | {x['etablissement_postal']}", axis=1),
    palette='viridis'
)
plt.title('Top (Origin-Destination, Current Etablissement) Pairs by Package Counts')
plt.xlabel('Number of Packages')
plt.ylabel('(Origin_Destination | Current Etablissement)')
plt.tight_layout()
plt.show()


### b. parsed_receptacles_df

In [ ]:
# For parsed_receptacles_dffffff: visualize the histogram of counts by (origin_destination, etablissement_postal) pairs

# First, select top N most common (origin_destination, etablissement_postal) pairs for clarity
pair_counts_current_receptacle = parsed_receptacles_df.groupby(['origin_destination', 'etablissement_postal']).size().reset_index(name='count')

# Sort and keep only top N pairs (by count)
top_n = 20
top_pairs_current_receptacle = pair_counts_current_receptacle.sort_values('count', ascending=False).head(top_n)

plt.figure(figsize=(16, 8))
sns.barplot(
    data=top_pairs_current_receptacle,
    x='count',
    y=top_pairs_current_receptacle.apply(lambda x: f"{x['origin_destination']} | {x['etablissement_postal']}", axis=1),
    palette='viridis'
)
plt.title('Top (Origin-Destination, Current Etablissement) Pairs by Receptacle Counts')
plt.xlabel('Number of Receptacles')
plt.ylabel('(Origin_Destination | Current Etablissement)')
plt.tight_layout()
plt.show()


we can see that the ETAB0002 is dominating and we remark that when the destination is DZ
we'll try to confirm that by taking into consideration the destination only


In [ ]:
# For parsed_receptacles_dffff: visualize the histogram of counts by (destination_country, etablissement_postal) pairs

# First, select top N most common (destination_country, etablissement_postal) pairs for clarity
pair_counts_dest_receptacle = parsed_receptacles_df.groupby(['destination_country', 'etablissement_postal']).size().reset_index(name='count')

# Sort and keep only top N pairs (by count)
top_n = 20
top_pairs_dest_receptacle = pair_counts_dest_receptacle.sort_values('count', ascending=False).head(top_n)

plt.figure(figsize=(16, 8))
sns.barplot(
    data=top_pairs_dest_receptacle,
    x='count',
    y=top_pairs_dest_receptacle.apply(lambda x: f"{x['destination_country']} | {x['etablissement_postal']}", axis=1),
    palette='viridis'
)
plt.title('Top (Destination Country, Current Etablissement) Pairs by Receptacle Counts')
plt.xlabel('Number of Receptacles')
plt.ylabel('(Destination Country | Current Etablissement)')
plt.tight_layout()
plt.show()


### This is to test the origin

In [ ]:
# For parsed_receptacles_dffffff: visualize the histogram of counts by (origin_country, etablissement_postal) pairs

# First, select top N most common (origin_country, etablissement_postal) pairs for clarity
pair_counts_origin_receptacle = parsed_receptacles_df.groupby(['origin_country', 'etablissement_postal']).size().reset_index(name='count')

# Sort and keep only top N pairs (by count)
top_n = 20
top_pairs_origin_receptacle = pair_counts_origin_receptacle.sort_values('count', ascending=False).head(top_n)

plt.figure(figsize=(16, 8))
sns.barplot(
    data=top_pairs_origin_receptacle,
    x='count',
    y=top_pairs_origin_receptacle.apply(lambda x: f"{x['origin_country']} | {x['etablissement_postal']}", axis=1),
    palette='viridis'
)
plt.title('Top (Origin Country, Current Etablissement) Pairs by Receptacle Counts')
plt.xlabel('Number of Receptacles')
plt.ylabel('(Origin Country | Current Etablissement)')
plt.tight_layout()
plt.show()


most of the values with ETAB0002 values are european countries in addition to AE and China(CN)

## 2. Next etablissement

### a. parsed_packages_df

In [ ]:
# For parsed_packages_df: visualize the histogram of counts by (origin_destination, next_etablissement_postal)

# First, select top N most common (origin_destination, next_etablissement_postal) pairs for clarity
pair_counts = parsed_packages_df.groupby(['origin_destination', 'next_etablissement_postal']).size().reset_index(name='count')

# Sort and keep only top N pairs (by count)
top_n = 20
top_pairs = pair_counts.sort_values('count', ascending=False).head(top_n)

plt.figure(figsize=(16, 8))
sns.barplot(
    data=top_pairs,
    x='count',
    y=top_pairs.apply(lambda x: f"{x['origin_destination']} | {x['next_etablissement_postal']}", axis=1),
    palette='viridis'
)
plt.title('Top (Origin-Destination, Next Etablissement) Pairs by Package Counts')
plt.xlabel('Number of Packages')
plt.ylabel('(Origin_Destination | Next Etablissement)')
plt.tight_layout()
plt.show()


### b. parsed_receptacles_df

In [ ]:
# For parsed_receptacles_dffff: visualize the histogram of counts by (origin_destination, next_etablissement_postal)

# First, select top N most common (origin_destination, next_etablissement_postal) pairs for clarity
pair_counts_recept = parsed_receptacles_df.groupby(['origin_destination', 'next_etablissement_postal']).size().reset_index(name='count')

# Sort and keep only top N pairs (by count)
top_n = 20
top_pairs_recept = pair_counts_recept.sort_values('count', ascending=False).head(top_n)

plt.figure(figsize=(16, 8))
sns.barplot(
    data=top_pairs_recept,
    x='count',
    y=top_pairs_recept.apply(lambda x: f"{x['origin_destination']} | {x['next_etablissement_postal']}", axis=1),
    palette='viridis'
)
plt.title('Top (Origin-Destination, Next Etablissement) Pairs by Receptacle Counts')
plt.xlabel('Number of Receptacles')
plt.ylabel('(Origin_Destination | Next Etablissement)')
plt.tight_layout()
plt.show()



Do for origin and for destination separately

In [ ]:

# For parsed_receptacles_df: visualize the histogram of counts by destination_country and next_etablissement_postal

# First, select top N most common (destination_country, next_etablissement_postal) pairs for clarity
pair_counts_dest = parsed_receptacles_df.groupby(['destination_country', 'next_etablissement_postal']).size().reset_index(name='count')

# Sort and keep only top N pairs (by count)
top_n = 20
top_pairs_dest = pair_counts_dest.sort_values('count', ascending=False).head(top_n)

plt.figure(figsize=(16, 8))
sns.barplot(
    data=top_pairs_dest,
    x='count',
    y=top_pairs_dest.apply(lambda x: f"{x['destination_country']} | {x['next_etablissement_postal']}", axis=1),
    palette='viridis'
)
plt.title('Top (Destination Country, Next Etablissement) Pairs by Receptacle Counts')
plt.xlabel('Number of Receptacles')
plt.ylabel('(Destination_Country | Next Etablissement)')
plt.tight_layout()
plt.show()


In [ ]:
# For parsed_receptacles_df: visualize the histogram of counts by origin_country and next_etablissement_postal

# First, select top N most common (origin_country, next_etablissement_postal) pairs for clarity
pair_counts_origin = parsed_receptacles_df.groupby(['origin_country', 'next_etablissement_postal']).size().reset_index(name='count')

# Sort and keep only top N pairs (by count)
top_n = 20
top_pairs_origin = pair_counts_origin.sort_values('count', ascending=False).head(top_n)

plt.figure(figsize=(16, 8))
sns.barplot(
    data=top_pairs_origin,
    x='count',
    y=top_pairs_origin.apply(lambda x: f"{x['origin_country']} | {x['next_etablissement_postal']}", axis=1),
    palette='viridis'
)
plt.title('Top (Origin Country, Next Etablissement) Pairs by Receptacle Counts')
plt.xlabel('Number of Receptacles')
plt.ylabel('(Origin_Country | Next Etablissement)')
plt.tight_layout()
plt.show()


from the analysis we can see that there are some etablissments that get congested forming a sort of loop (ETAB0030, ETAB0002, ETAB0006)

### Time analysis regarding origin

### 1. parsed_packages_df

In [ ]:
# Group the parsed_packages_df by date and origin_country to count packages per month per origin country
parsed_packages_df['date'] = pd.to_datetime(parsed_packages_df['date'])

# Group by origin_country and resample by month
ts_by_origin = parsed_packages_df.groupby(['origin_country', pd.Grouper(key='date', freq='ME')])['MAILITM_FID'].count().reset_index()

# Choose top 5 origin countries by total count for plotting
top_origin = parsed_packages_df['origin_country'].value_counts().head(5).index
plt.figure(figsize=(14, 7))

for origin in top_origin:
    ts = ts_by_origin[ts_by_origin['origin_country'] == origin]
    plt.plot(ts['date'], ts['MAILITM_FID'], marker='o', label=origin)

plt.title('Monthly Package count by Origin Country')
plt.xlabel('Date')
plt.ylabel('Number of Packages')
plt.legend(title='Origin Country')
plt.tight_layout()
plt.show()


### 2. parsed_receptacles_dfffff

In [ ]:
# Group the parsed_receptacles_df by date and origin_country to count receptacles per month per origin country
parsed_receptacles_df['date'] = pd.to_datetime(parsed_receptacles_df['date'])

# Group by origin_country and resample by month
ts_by_origin_recept = parsed_receptacles_df.groupby(['origin_country', pd.Grouper(key='date', freq='ME')])['RECPTCL_FID'].count().reset_index()

# Choose top 5 origin countries by total count for plotting
top_origin_recept = parsed_receptacles_df['origin_country'].value_counts().head(5).index
plt.figure(figsize=(14, 7))

for origin in top_origin_recept:
    ts = ts_by_origin_recept[ts_by_origin_recept['origin_country'] == origin]
    plt.plot(ts['date'], ts['RECPTCL_FID'], marker='o', label=origin)

plt.title('Monthly Receptacle count by Origin Country')
plt.xlabel('Date')
plt.ylabel('Number of Receptacles')
plt.legend(title='Origin Country')
plt.tight_layout()
plt.show()


### Time analysis by destination

In [ ]:
# Group the parsed_packages_df by date and destination_country to count packages per month per destination country
parsed_packages_df['date'] = pd.to_datetime(parsed_packages_df['date'])

# Group by destination_country and resample by month
ts_by_dest = parsed_packages_df.groupby(['destination_country', pd.Grouper(key='date', freq='ME')])['MAILITM_FID'].count().reset_index()

# Choose top 5 destination countries by total count for plotting
top_dest = parsed_packages_df['destination_country'].value_counts().head(5).index
plt.figure(figsize=(14, 7))

for dest in top_dest:
    ts = ts_by_dest[ts_by_dest['destination_country'] == dest]
    plt.plot(ts['date'], ts['MAILITM_FID'], marker='o', label=dest)

plt.title('Monthly Package count by Destination Country')
plt.xlabel('Date')
plt.ylabel('Number of Packages')
plt.legend(title='Destination Country')
plt.tight_layout()
plt.show()


### 2. parsed_receptacles_dffff

In [ ]:
# Group the parsed_receptacles_df by date and destination_country to count receptacles per month per destination country
parsed_receptacles_df['date'] = pd.to_datetime(parsed_receptacles_df['date'])

# Group by destination_country and resample by month
ts_by_dest_recept = parsed_receptacles_df.groupby(['destination_country', pd.Grouper(key='date', freq='ME')])['RECPTCL_FID'].count().reset_index()

# Choose top 5 destination countries by total count for plotting
top_dest_recept = parsed_receptacles_df['destination_country'].value_counts().head(5).index
plt.figure(figsize=(14, 7))

for dest in top_dest_recept:
    ts = ts_by_dest_recept[ts_by_dest_recept['destination_country'] == dest]
    plt.plot(ts['date'], ts['RECPTCL_FID'], marker='o', label=dest)

plt.title('Monthly Receptacle count by Destination Country')
plt.xlabel('Date')
plt.ylabel('Number of Receptacles')
plt.legend(title='Destination Country')
plt.tight_layout()
plt.show()


## Creating additional features
 1. 'flow_type' column with values: 'inbound' (to DZ), 'outbound' (from DZ), 'local' (DZ to DZ), otherwise 'other'

In [ ]:
import numpy as np

def get_flow_type(df):
    # Define the conditions
    conditions = [
        (df['destination_country'] == 'DZ') & (df['origin_country'] == 'DZ'), # local
        (df['destination_country'] == 'DZ'),                                # inbound
        (df['origin_country'] == 'DZ')                                     # outbound
    ]

    # Define the results for each condition
    choices = ['local', 'inbound', 'outbound']

    # Apply logic with 'other' as the default
    return np.select(conditions, choices, default='other')

# Apply to both DataFrames instantly
parsed_packages_df['flow_type'] = get_flow_type(parsed_packages_df)
parsed_receptacles_df['flow_type'] = get_flow_type(parsed_receptacles_df)

# Print counts
print("Flow type counts in parsed_packages_df:\n", parsed_packages_df['flow_type'].value_counts())
print("\nFlow type counts in parsed_receptacles_df:\n", parsed_receptacles_df['flow_type'].value_counts())

we can see that there are some values of flow type with the type "other"

In [ ]:
# Display the first 10 rows of origin_country and destination_country for flow_type 'other' in parsed_receptacles_df
print(parsed_receptacles_df.loc[parsed_receptacles_df['flow_type'] == 'other', ['origin_country', 'destination_country']].head(10))



## Analysis of the relation between the flow_type and the event_type

## 1. parsed_packages_df

In [ ]:

# Analyse relation between flow_type and EVENT_TYPE_CD in parsed_packages_df

plt.figure(figsize=(14,7))
ax = sns.countplot(data=parsed_packages_df, x="EVENT_TYPE_CD", hue="flow_type", palette='Set2')
plt.title("Distribution of EVENT_TYPE_CD by flow_type in parsed_packages_df")
plt.xlabel("EVENT_TYPE_CD")
plt.ylabel("Count")
plt.legend(title="Flow Type")
plt.tight_layout()
for c in ax.containers:
    ax.bar_label(c, label_type='edge', fontsize=8, padding=2)
plt.show()



## 2. parsed_receptacles_df

In [ ]:
# For each flow_type, list the unique EVENT_TYPE_CD values and the most frequent EVENT_TYPE_CD value
for flow in parsed_receptacles_df['flow_type'].unique():
    event_types = parsed_receptacles_df.loc[parsed_receptacles_df['flow_type'] == flow, 'EVENT_TYPE_CD'].unique()
    most_common_event_type = parsed_receptacles_df.loc[parsed_receptacles_df['flow_type'] == flow, 'EVENT_TYPE_CD'].mode()
    print(f"Flow type: {flow}")
    print(f"EVENT_TYPE_CD values: {sorted(event_types)}")
    if not most_common_event_type.empty:
        print(f"Most frequent EVENT_TYPE_CD: {most_common_event_type.iloc[0]}")
    else:
        print("No EVENT_TYPE_CD available")
    print()


In [ ]:
plt.figure(figsize=(14,7))
ax = sns.countplot(data=parsed_receptacles_df, x="EVENT_TYPE_CD", hue="flow_type", palette='Set1')
plt.title("Distribution of EVENT_TYPE_CD by flow_type in parsed_receptacles_df")
plt.xlabel("EVENT_TYPE_CD")
plt.ylabel("Count")
plt.legend(title="Flow Type")
plt.tight_layout()

for c in ax.containers:
    ax.bar_label(c, label_type='edge', fontsize=8, padding=2)

plt.show()


we can see that there are major event types related to the inbound flow type( coming to DZ)

# Track multiple receptacles just to see the flow of events

In [ ]:
# Track multiple receptacles: visualize all events for 5 different RECPTCL_FID in parsed_receptacles_df
# Pick 5 unique RECPTCL_FID values to demonstrate
num_examples = 5
example_receptacle_ids = parsed_receptacles_df['RECPTCL_FID'].drop_duplicates().iloc[:num_examples]
for rid in example_receptacle_ids:
    print("\n--- Events for RECPTCL_FID:", rid, "---")
    display(parsed_receptacles_df[parsed_receptacles_df['RECPTCL_FID'] == rid][['RECPTCL_FID', 'date', 'EVENT_TYPE_CD', 'etablissement_postal', 'next_etablissement_postal']].sort_values('date').reset_index(drop=True))


plt.figure(figsize=(14, 5))

for i, receptacle_id in enumerate(example_receptacle_ids):
    ex_df = parsed_receptacles_df[parsed_receptacles_df['RECPTCL_FID'] == receptacle_id].sort_values('date')
    plt.plot(
        ex_df['date'],
        ex_df['EVENT_TYPE_CD'],
        marker='o',
        label=f"RECPTCL_FID: {receptacle_id}"
    )

plt.xticks(rotation=45)
plt.title(f"Event Timeline (EVENT_TYPE_CD) for {num_examples} Receptacles")
plt.xlabel("Date")
plt.ylabel("EVENT_TYPE_CD")
plt.tight_layout()
plt.legend()
plt.show()


In [ ]:
packages_df=parsed_packages_df.copy()
receptacles_df=parsed_receptacles_df.copy()

In [ ]:
packages_df['EVENT_TYPE_CD'] = packages_df['EVENT_TYPE_CD'].astype(str)
receptacles_df['EVENT_TYPE_CD'] = receptacles_df['EVENT_TYPE_CD'].astype(str)

In [ ]:
packages_df['month'] = packages_df['date'].dt.month
receptacles_df['month'] = receptacles_df['date'].dt.month
packages_df['hour'] = packages_df['date'].dt.hour
receptacles_df['hour'] = receptacles_df['date'].dt.hour
packages_df['day_of_week'] = packages_df['date'].dt.day_name()
receptacles_df['day_of_week'] = receptacles_df['date'].dt.day_name()

In [ ]:
clus_packages_df = packages_df.copy()
clus_receptacles_df = receptacles_df.copy()

# **START OF MODELING**

- we need to delete the first record of each package to not take into consideration the time it was inside its receptacle since this time would be calculated separatly

In [ ]:
def drop_first_pkg_row(df):
    # keep='first' marks the first occurrence of each ID as False
    # Subsequent occurrences are marked as True
    is_not_first_row = df.duplicated(subset=['MAILITM_FID'], keep='first')

    # Return the filtered dataframe WITHOUT resetting the index
    return df[is_not_first_row]

In [ ]:
packages_df = drop_first_pkg_row(packages_df)

In [ ]:
packages_df.head()

### Adding the target `route_duration`
* In the following cell, we're adding the target `route_duration` for each row.
* for rows (of the same package/receptacle) having `next_etablissement_postal` different than `etablissement_postal` of their next row, we keep the value of `route_duration` NaN.
* otherwise, we get the difference of `date` of the row and its next (of the same package/receptacle) in `hours` and store it in `route_duration`.
- **NOTE**: We'll drop the first row of each package before adding the target `route_duration`

In [ ]:
def add_target(df, id):

    # 1. Sort the entire dataset chronologically
    df = df.sort_values(by=[id, 'date'], ascending=True)

    # ... ADD THE route_duration TARGET ...

    df['next_event_date'] = df.groupby(id)['date'].shift(-1)
    df['route_duration'] = (df['next_event_date'] - df['date']).dt.total_seconds() / 3600
    # Create logical consistency mask m
    df['next_row_etab'] = df.groupby(id)['etablissement_postal'].shift(-1)
    valid_route_duration_mask = (df['next_etablissement_postal'] == df['next_row_etab'])

    df.loc[~valid_route_duration_mask, 'route_duration'] = np.nan
    df = df.drop(columns=['next_event_date', 'next_row_etab']) # to keep only to the original features.

    return df


In [ ]:
packages_df = add_target(packages_df, 'MAILITM_FID')
receptacles_df = add_target(receptacles_df, 'RECPTCL_FID')

In [ ]:
import numpy as np

In [ ]:
large_duration_pkg = packages_df[packages_df['route_duration'] > 1000]
large_duration_pkg.sort_values('route_duration', ascending=False).head()

In [ ]:
large_duration_rcptcl = receptacles_df[receptacles_df['route_duration'] > 1000]
large_duration_rcptcl.sort_values('route_duration', ascending=False).head()

### Handling null values in the target

In [ ]:
packages_df[packages_df['route_duration'].isna()].shape[0] / packages_df.shape[0] * 100

* 75% of the rows of `packages_df` have null target, these will be dropped since target values needs to be **correct** and not imputed or guessed.

In [ ]:
packages_df = packages_df[~packages_df['route_duration'].isna()]
packages_df.shape, packages_df['next_etablissement_postal'].isna().any()

* For `receptacles_df`:

In [ ]:
receptacles_df[receptacles_df['route_duration'].isna()].shape[0] / receptacles_df.shape[0] * 100

* 77% of the rows of `receptacles_df` have null target, these will be dropped since target values needs to be **correct** and not imputed or guessed.

In [ ]:
receptacles_df = receptacles_df[~receptacles_df['route_duration'].isna()]
receptacles_df.shape, receptacles_df['next_etablissement_postal'].isna().any()

### Before splitting We add :
  - `packages_per_receptacle`: number of packages per receptacle

In [ ]:
def add_packages_per_receptacle(receptacles_df, packages_df):
    # 1. Calculate counts directly from packages
    package_counts = packages_df.groupby('RECPTCL_FID')['MAILITM_FID'].count().reset_index()
    package_counts.columns = ['RECPTCL_FID', 'packages_per_receptacle']

    # 2. Merge once onto the receptacles dataframe
    receptacles_df = pd.merge(receptacles_df, package_counts, on='RECPTCL_FID', how='left')

    # 3. Fill empty receptacles with 0
    receptacles_df['packages_per_receptacle'] = receptacles_df['packages_per_receptacle'].fillna(0)

    return receptacles_df

In [ ]:
receptacles_df = add_packages_per_receptacle(receptacles_df, packages_df)

# Splitting

- Since the data is dependent with time we need to take that into consideration and split into past and future

In [ ]:
def my_train_test_split(df):
    df = df.sort_values('date')

    split_idx = int(len(df) * 0.8)

    train_df = df.iloc[:split_idx]
    test_df = df.iloc[split_idx:]

    return train_df, test_df

* Splitting `packages` and dataset into training and testing sets using `train_test_split`

In [ ]:

pkg_X_train, pkg_X_test= my_train_test_split(packages_df)
print (f"Training set size: {pkg_X_train.shape[0]} rows")
print (f"Testing set size: {pkg_X_test.shape[0]} rows")

* Splitting `receptacles` and dataset into training and testing sets using `train_test_split`

In [ ]:
rcp_X_train, rcp_X_test= my_train_test_split(receptacles_df)
print (f"Training set size: {rcp_X_train.shape[0]} rows")
print (f"Testing set size: {rcp_X_test.shape[0]} rows")

# Adding New features
  

### Dynamic Route load and Etablissement load
* We'll add two features `route_load_1h` and `ETAB_load_1h` which for each row, calculates the number of packages/receptacles passing by a specific route (ETAB_XXXX -> ETAB_YYYY) and the number of packages/receptacles at a specific ETAB_XXXX for the previous 1h, respectively.

In [ ]:
def calculate_etab_load_1h(df,id_col):
    # 1. Store the original order/index
    df = df.copy()
    df['original_index'] = df.index

    # 2. Sort by etab and date for the rolling calculation
    # We keep the unique original_index to map values back correctly
    df_sorted = df.sort_values(['etablissement_postal', 'date'])

    # 3. Calculate rolling count
    # We use 'date' as the window, but we keep it in the index alongside the original_index
    rolling_series = (
        df_sorted.set_index('date')
        .groupby('etablissement_postal')[id_col]
        .rolling('1h', closed='left')
        .count()
    )

    # 4. Map it back safely
    # We reset the index of rolling_series to get a flat dataframe
    # Then we align it back to the original dataframe
    rolling_df = rolling_series.reset_index()

    # Since rolling and groupby can reorder rows, we merge on
    # the specific columns to ensure every package gets its correct count
    # To handle duplicates, we add a temporary 'sequence' within each millisecond
    df_sorted['temp_seq'] = df_sorted.groupby(['etablissement_postal', 'date']).cumcount()
    rolling_df['temp_seq'] = rolling_df.groupby(['etablissement_postal', 'date']).cumcount()

    df_final = pd.merge(
        df_sorted,
        rolling_df.rename(columns={id_col: 'etab_load_1h'}),
        on=['etablissement_postal', 'date', 'temp_seq'],
        how='left'
    )
    df_final['etab_load_1h'] = df_final['etab_load_1h'].fillna(0)

    return df_final.drop(columns=['temp_seq', 'original_index'])

In [ ]:
def calculate_route_load_1h(df,id_col):
    df = df.copy()

    # 1. Sort by route and date
    # We use next_etablissement_postal which you've already filled
    df_sorted = df.sort_values(['etablissement_postal', 'next_etablissement_postal', 'date'])

    # 2. Calculate rolling count for the specific LANE
    rolling_series = (
        df_sorted.set_index('date')
        .groupby(['etablissement_postal', 'next_etablissement_postal'])[id_col]
        .rolling('1h', closed='left')
        .count()
    )

    # 3. Flatten and prepare for merge
    rolling_df = rolling_series.reset_index()

    # Handle duplicates with sequence counts
    df_sorted['temp_seq'] = df_sorted.groupby(['etablissement_postal', 'next_etablissement_postal', 'date']).cumcount()
    rolling_df['temp_seq'] = rolling_df.groupby(['etablissement_postal', 'next_etablissement_postal', 'date']).cumcount()

    # 4. Merge back
    df_final = pd.merge(
        df_sorted,
        rolling_df.rename(columns={id_col: 'route_load_1h'}),
        on=['etablissement_postal', 'next_etablissement_postal', 'date', 'temp_seq'],
        how='left'
    )

    # Fill NaNs with 0 (No other packages on this lane in the last 1h)
    df_final['route_load_1h'] = df_final['route_load_1h'].fillna(0)

    return df_final.drop(columns=['temp_seq'])

### Time since first scan
- The difference between the time of the actual scan with the first scan this will be usefull when giving the total duration

In [ ]:
def add_time_since_first_scan(df, id_col):
    # Ensure we are looking at the same entity in the right order
    df = df.sort_values([id_col, 'date'])

    # 1. TIME SINCE FIRST SCAN
    # Subtract the minimum date of the group from the current date
    df['time_since_first_scan'] = df.groupby(id_col)['date'].transform(
        lambda x: (x - x.min()).dt.total_seconds() / 3600
    )

    return df

- The difference of time between the actual scan and its previous one 

In [ ]:
def add_time_since_last_scan(df,id_col):
    # Ensure we are looking at the same package in the right order
    df = df.sort_values([id_col, 'date'])

    # 1. TIME SINCE LAST SCAN
    # This is NOT cumulative.
    df['time_since_last_scan'] = df.groupby(id_col)['date'].diff().dt.total_seconds() / 3600
    df['time_since_last_scan'] = df['time_since_last_scan'].fillna(0)
    return df


In [ ]:
for df in [pkg_X_train, pkg_X_test]:
    # 4 = Friday, 5 = Saturday
    df['is_weekend'] = df['date'].dt.dayofweek.isin([4, 5]).astype(int)
    df['first_last_week_day']= df['date'].dt.dayofweek.isin([3,6]).astype(int)

- This adds more meaning to the service_indicator to make a relation with the origin country

In [ ]:
pkg_X_train['country_service'] = pkg_X_train['origin_country'].astype(str) + "_" + pkg_X_train['service_indicator'].astype(str)
pkg_X_test['country_service'] = pkg_X_test['origin_country'].astype(str) + "_" + pkg_X_test['service_indicator'].astype(str)

- Adding features that represents events that could affect the flow of packages in algeria ``Note that this is only for packages because they are the ones that are related to Algeria``

In [ ]:
import holidays

# 1. Setup Algeria Holidays (Keep this outside the function for speed)
al_holidays = holidays.Algeria(years=[2023, 2024, 2025, 2026])
holiday_dates = sorted(al_holidays.keys())
holiday_df = pd.DataFrame({'holiday_date': pd.to_datetime(holiday_dates)})

def add_holidays_features(df):
    # Ensure Date is datetime and SORTED (merge_asof requires sorting)
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values('date').copy() # .copy() avoids SettingWithCopy warnings

    # 3. Calculate "Days Since Last Holiday"
    # direction='backward' looks for the last holiday <= current date
    df = pd.merge_asof(df, holiday_df, left_on='date', right_on='holiday_date', direction='backward')
    df['days_since_last_holiday'] = (df['date'] - df['holiday_date']).dt.days
    df = df.drop(columns=['holiday_date']) # Drop it so the next merge doesn't conflict

    # 4. Calculate "Days Until Next Holiday"
    # direction='forward' looks for the next holiday >= current date
    df = pd.merge_asof(df, holiday_df, left_on='date', right_on='holiday_date', direction='forward')
    df['days_until_next_holiday'] = (df['holiday_date'] - df['date']).dt.days
    df = df.drop(columns=['holiday_date'])

    # 5. Clean up and Cap
    df[['days_since_last_holiday', 'days_until_next_holiday']] = df[['days_since_last_holiday', 'days_until_next_holiday']].fillna(30)

    df['days_since_last_holiday'] = df['days_since_last_holiday'].clip(upper=30)
    df['days_until_next_holiday'] = df['days_until_next_holiday'].clip(upper=30)

    return df

In [ ]:
pkg_X_train = add_holidays_features(pkg_X_train)
pkg_X_test = add_holidays_features(pkg_X_test)

- This will be tested later to see whether we can add it to receptacles or not(additional information are needed to be applied since we don't know if the receptacle is still at the origin country or at the destination)

In [ ]:
# import pandas as pd
# import holidays

# # 1. Pre-generate holidays for all required countries
# # we can use the Countries_codes given in order to create the list of supported countries for holidays
# SUPPORTED_COUNTRIES = [TO BE FILLED]
# all_holidays = []

# for country in SUPPORTED_COUNTRIES:
#     # Get holidays for each country
#     country_hols = holidays.CountryHoliday(country, years=[2023, 2024, 2025, 2026])
#     for date, name in country_hols.items():
#         all_holidays.append({'holiday_date': pd.to_datetime(date), 'country_code': country})

# # Create the master holiday DataFrame
# holiday_df = pd.DataFrame(all_holidays).sort_values('holiday_date')

# def add_holidays_features(df, country_col='country_name'):
#     """
#     df: input dataframe
#     country_col: the column containing the country name (must match holidays package names)
#     """
#     df['date'] = pd.to_datetime(df['date'])
#     df = df.sort_values('date').copy()

#     # We will process each country group separately and concat them back
#     processed_groups = []

#     for country, group in df.groupby(country_col):
#         # Filter holiday_df for just this country
#         country_hols = holiday_df[holiday_df['country_code'] == country]

#         if country_hols.empty:
#             # If country not in holidays, set defaults immediately
#             group['days_since_last_holiday'] = 30
#             group['days_until_next_holiday'] = 30
#         else:
#             # Days Since Last Holiday
#             group = pd.merge_asof(group, country_hols, left_on='date', right_on='holiday_date', direction='backward')
#             group['days_since_last_holiday'] = (group['date'] - group['holiday_date']).dt.days
#             group = group.drop(columns=['holiday_date', 'country_code'])

#             # Days Until Next Holiday
#             group = pd.merge_asof(group, country_hols, left_on='date', right_on='holiday_date', direction='forward')
#             group['days_until_next_holiday'] = (group['holiday_date'] - group['date']).dt.days
#             group = group.drop(columns=['holiday_date', 'country_code'])

#         processed_groups.append(group)

#     # Combine back into one dataframe
#     df_final = pd.concat(processed_groups)

#     # 5. Clean up and Cap (same as your original logic)
#     cols = ['days_since_last_holiday', 'days_until_next_holiday']
#     df_final[cols] = df_final[cols].fillna(30).clip(upper=30)

#     return df_final

### Creating the final function to create all the shared features at once

In [ ]:
def create_all_shared_features(df, id_col):
    df = calculate_etab_load_1h(df, id_col)
    df = calculate_route_load_1h(df, id_col)
    df = add_time_since_first_scan(df, id_col)
    df = add_time_since_last_scan(df, id_col)
    return df

In [ ]:
pkg_X_train = create_all_shared_features(pkg_X_train, 'MAILITM_FID')
pkg_X_test = create_all_shared_features(pkg_X_test, 'MAILITM_FID')
rcp_X_train = create_all_shared_features(rcp_X_train, 'RECPTCL_FID')
rcp_X_test = create_all_shared_features(rcp_X_test, 'RECPTCL_FID')

- check for features creation

In [ ]:
pkg_X_train.columns

In [ ]:
rcp_X_train.columns

# Use CatBoost

 ## 1. `For packages`

### Use Catboost with Huber loss function
- Huber with delta=20 hours act as MAE for the ones with less than 20 hours and act as RMSE for the ones greater than delta.
- This will solve the obssession of RMSE toward predicting the outliers(large values)
- parameters are set manually

- This could be run for testing since the hyperparameter tuning would take time(note that the parameters used are the ones given by the hyperparameters tuning)

In [ ]:
# !pip install catboost

In [ ]:
import numpy as np
from catboost import CatBoostRegressor, Pool, cv

# 1. Ensure chronological order
pkg_X_train = pkg_X_train.sort_values('date').reset_index(drop=True)
pkg_X_test = pkg_X_test.sort_values('date').reset_index(drop=True)

# --- Define Model Parameters and Features ---
TARGET_COL = 'route_duration'

cat_features = [
    'etablissement_postal',
    'next_etablissement_postal',
    'day_of_week',
    'service_indicator',
    'origin_destination',
    'EVENT_TYPE_CD',
    'hour',
    'month',
    'is_weekend',
    'first_last_week_day',
    'country_service'
]

# Columns to drop
drop_cols = [TARGET_COL, 'MAILITM_FID', 'RECPTCL_FID', 'date', 'serial_number',
             'flow_type', 'country_code', 'destination_country', 'origin_country']

# Prepare the full 80% training set
X_train_full = pkg_X_train.drop(columns=[c for c in drop_cols if c in pkg_X_train.columns])
y_train_full = pkg_X_train[TARGET_COL]

# Identify categorical features by name
categorical_feature_names = [c for c in cat_features if c in X_train_full.columns]

# --- TimeSeries Cross-Validation to Find Optimal Iterations ---

# Create the training Pool
train_pool = Pool(
    data=X_train_full,
    label=y_train_full,
    cat_features=categorical_feature_names,
)

cv_params = {
    'loss_function': 'Huber:delta=20.0',          #Hurber(acts as MAE for normal ones but acts as RMSE for outliers)
    'eval_metric': 'MAE',
    'iterations': 1500,
    'learning_rate': 0.014927636750123532,
    'depth': 8,
    'random_seed': 42,
    'verbose': 0,
    'l2_leaf_reg': 8.27674007602611,
}

print("Starting TimeSeries Cross-Validation (5 Folds) optimizing for MAE...")

cv_results = cv(
    params=cv_params,
    pool=train_pool,
    fold_count=5,
    shuffle=False,
    type='TimeSeries',
)

# Find the best iteration based on the minimum average MAE
# Note: cv_results keys change based on the loss_function used
best_iter = cv_results['test-MAE-mean'].values.argmin()
best_mae = cv_results['test-MAE-mean'].min()
print(f"Optimal number of CatBoost iterations: {best_iter + 1} (Best CV MAE: {best_mae:.4f})")

# --- Train Final Model on ENTIRE Training Set ---

final_model = CatBoostRegressor(
    iterations=best_iter + 1,
    learning_rate=0.014927636750123532,
    depth=8,
    loss_function='Huber:delta=20.0',
    eval_metric='MAE',
    random_seed=42,
    cat_features=categorical_feature_names,
    verbose=100
)

print("\nTraining final model on the entire 80% training set...")
final_model.fit(train_pool)
#saving the model
final_model.save_model("../models/final_pkg_route_duration_model.cbm")
print("Model trained and saved as 'final_pkg_route_duration_model.cbm'")
# --- Predict and Evaluate on the Future Test Set ---

X_test_features = pkg_X_test.drop(columns=[c for c in drop_cols if c in pkg_X_test.columns])
y_test_target = pkg_X_test[TARGET_COL]

predictions = final_model.predict(X_test_features)


### Using Optuna for hyperparameters tuning before passing them to CatBoost

- Preparing the data pool
- Uncomment to run (would taake time)

In [ ]:
# from catboost import Pool

# # ---  Data Preparation ---
# pkg_X_train = pkg_X_train.sort_values('date').reset_index(drop=True)
# pkg_X_test = pkg_X_test.sort_values('date').reset_index(drop=True)
# TARGET_COL = 'route_duration'
# cat_features = [
#     'etablissement_postal', 'next_etablissement_postal', 'day_of_week',
#     'service_indicator', 'origin_destination', 'EVENT_TYPE_CD', 'hour', 'month',
#     'is_weekend', 'first_last_week_day', 'service_country'
# ]
# drop_cols = [TARGET_COL, 'MAILITM_FID', 'RECPTCL_FID', 'date', 'serial_number',
#              'flow_type', 'country_code', 'destination_country', 'origin_country']

# X_train_full = pkg_X_train.drop(columns=[c for c in drop_cols if c in pkg_X_train.columns])
# y_train_full = pkg_X_train[TARGET_COL]
# categorical_feature_names = [c for c in cat_features if c in X_train_full.columns]

# train_pool = Pool(data=X_train_full, label=y_train_full, cat_features=categorical_feature_names)


- This will find the best parameters and saves them into a json file so they can be used later without the need to do the tuning again unless changes are made in the data or the features

In [ ]:
# !pip install optuna

In [ ]:
# import optuna
# import json
# from catboost import cv
# def objective(trial):
#     params = {
#         'loss_function': 'Huber:delta=20.0',
#         'eval_metric': 'MAE',
#         'custom_metric': ['RMSE'], # Ensure RMSE is calculated
#         'random_seed': 42,
#         'verbose': 0,
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
#         'depth': trial.suggest_int('depth', 4, 8),
#         'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
#         'iterations': 1000,
#         'early_stopping_rounds': 30
#     }

#     cv_results = cv(
#         params=params,
#         pool=train_pool,
#         fold_count=3,
#         shuffle=False,
#         type='TimeSeries',
#         plot=False
#     )

#     # Track both metrics
#     best_mae = cv_results['test-MAE-mean'].min()
#     best_rmse = cv_results['test-RMSE-mean'].min()

#     return best_mae, best_rmse

# # --- 2. Run Multi-Objective Optimization ---
# print("Starting Hyperparameter Tuning (Tracking MAE & RMSE)...")
# study = optuna.create_study(directions=['minimize', 'minimize'])
# study.optimize(objective, n_trials=20)

# # --- 3. Pick the best trial and find optimal iterations ---
# # We pick the trial with the lowest MAE
# best_trial = min(study.best_trials, key=lambda t: t.values[0])

# print(f"\nBest Trial selected (MAE: {best_trial.values[0]:.4f}, RMSE: {best_trial.values[1]:.4f})")

# # Find best iterations for the winning set on 5 folds for stability
# final_params_temp = {
#     'loss_function': 'Huber:delta=20.0',
#     'eval_metric': 'MAE',
#     **best_trial.params
# }

# print("Calculating final optimal iterations...")
# final_cv = cv(params=final_params_temp, pool=train_pool, fold_count=5, shuffle=False, type='TimeSeries', verbose=0)
# best_iteration = int(final_cv['test-MAE-mean'].values.argmin() + 1)

# # --- 4. Save to JSON ---
# best_config = {
#     **best_trial.params,
#     "iterations": best_iteration,
#     "loss_function": 'Huber:delta=20.0',
#     "eval_metric": 'MAE',
#     "final_mae": best_trial.values[0],
#     "final_rmse": best_trial.values[1]
# }

# with open("catboost_pkg_best_params.json", "w") as f:
#     json.dump(best_config, f, indent=4)

# print(f"Tuning complete. Best parameters saved to 'catboost_pkg_best_params.json'")

- Load the parameters from the json file and use them for the final training and then test

In [ ]:
# import json
# from catboost import CatBoostRegressor

# # --- 1. Load Parameters ---
# with open("catboost_pkg_best_params.json", "r") as f:
#     best_config = json.load(f)

# # Extract iterations and remove metrics/metadata so CatBoost doesn't crash
# iters = best_config.pop("iterations")
# best_config.pop("final_mae", None)   # Remove if present
# best_config.pop("final_rmse", None)  # Remove if present

# # --- 2. Initialize and Train Model ---
# # Ensure categorical_feature_names and train_pool are defined in your environment
# final_model = CatBoostRegressor(
#     iterations=iters,
#     **best_config,
#     cat_features=categorical_feature_names,
#     random_seed=42,
#     verbose=100
# )

# print("Training final model with saved parameters...")
# final_model.fit(train_pool)

# # --- 3. Save the actual Model weights ---
# final_model.save_model("final_pkg_route_duration_model.cbm")
# print("Model trained and saved as 'final_pkg_route_duration_model.cbm'")

# # --- 4. Predict on Test Set ---
# # Ensure pkg_X_test and drop_cols are defined in your environment
# X_test_features = pkg_X_test.drop(columns=[c for c in drop_cols if c in pkg_X_test.columns])
# predictions = final_model.predict(X_test_features)


 ## 2. `For receptacles`

### Use Catboost with Huber loss function

In [ ]:
import numpy as np
from catboost import CatBoostRegressor, Pool, cv

# 1. Ensure chronological order
rcp_X_train = rcp_X_train.sort_values('date').reset_index(drop=True)
rcp_X_test = rcp_X_test.sort_values('date').reset_index(drop=True)

# --- Define Model Parameters and Features ---
TARGET_COL = 'route_duration'

cat_features = [
    'etablissement_postal',
    'next_etablissement_postal',
    'day_of_week',
    'origin_destination',
    'EVENT_TYPE_CD',
    'hour',
    'month',
    'flow_type',
]

# Columns to drop
drop_cols = [TARGET_COL, 'RECPTCL_FID', 'date', 'destination_country', 'origin_country']

# Prepare the full 80% training set
X_train_full = rcp_X_train.drop(columns=[c for c in drop_cols if c in rcp_X_train.columns])
y_train_full = rcp_X_train[TARGET_COL]

# Identify categorical features by name
categorical_feature_names = [c for c in cat_features if c in X_train_full.columns]

# --- TimeSeries Cross-Validation to Find Optimal Iterations ---

# Create the training Pool
train_pool = Pool(
    data=X_train_full,
    label=y_train_full,
    cat_features=categorical_feature_names,
)

cv_params = {
    'loss_function': 'Huber:delta=20.0',          #  Hurber(acts as MAE for normal ones but acts as RMSE for outliers)
    'eval_metric': 'MAE',
    'iterations': 1500,
    'learning_rate': 0.08188269603595484,
    'depth': 5,
    'random_seed': 42,
    'verbose': 0,
    'l2_leaf_reg': 5.899785245396042,          # Standard regularization
}

print("Starting TimeSeries Cross-Validation (5 Folds) optimizing for MAE...")

cv_results = cv(
    params=cv_params,
    pool=train_pool,
    fold_count=5,
    shuffle=False,
    type='TimeSeries',
)

# Find the best iteration based on the minimum average MAE
# Note: cv_results keys change based on the loss_function used
best_iter = cv_results['test-MAE-mean'].values.argmin()
best_mae = cv_results['test-MAE-mean'].min()
print(f"Optimal number of CatBoost iterations: {best_iter + 1} (Best CV MAE: {best_mae:.4f})")

# --- Train Final Model on ENTIRE Training Set ---

final_model = CatBoostRegressor(
    iterations=best_iter + 1,
    learning_rate=0.08188269603595484,
    depth=5,
    loss_function='Huber:delta=20.0',
    eval_metric='MAE',
    random_seed=42,
    cat_features=categorical_feature_names,
    verbose=100
)

print("\nTraining final model on the entire 80% training set...")
final_model.fit(train_pool)
# --- 3. Save the actual Model weights ---
final_model.save_model("final_rcp_route_duration_model.cbm")
print("Model trained and saved as 'final_rcp_route_duration_model.cbm'")

# --- Predict and Evaluate on the Future Test Set ---

X_test_features = rcp_X_test.drop(columns=[c for c in drop_cols if c in rcp_X_test.columns])
y_test_target = rcp_X_test[TARGET_COL]

predictions = final_model.predict(X_test_features)


### Using Optuna for hyperparameters tuning
- Uncomment to run

In [ ]:
# from catboost import Pool

# # ---  Data Preparation ---
# rcp_X_train = rcp_X_train.sort_values('date').reset_index(drop=True)
# rcp_X_test = rcp_X_test.sort_values('date').reset_index(drop=True)
# TARGET_COL = 'route_duration'
# cat_features = [
#     'etablissement_postal',
#     'next_etablissement_postal',
#     'day_of_week',
#     'origin_destination',
#     'EVENT_TYPE_CD',
#     'hour',
#     'month',
#     'flow_type',
# ]
# drop_cols = [TARGET_COL, 'RECPTCL_FID', 'date', 'destination_country', 'origin_country']
# X_train_full = rcp_X_train.drop(columns=[c for c in drop_cols if c in rcp_X_train.columns])
# y_train_full = rcp_X_train[TARGET_COL]
# categorical_feature_names = [c for c in cat_features if c in X_train_full.columns]

# train_pool = Pool(data=X_train_full, label=y_train_full, cat_features=categorical_feature_names)


In [ ]:
# import optuna
# import json
# from catboost import cv
# def objective(trial):
#     params = {
#         'loss_function': 'Huber:delta=20.0',
#         'eval_metric': 'MAE',
#         'custom_metric': ['RMSE'], # Ensure RMSE is calculated
#         'random_seed': 42,
#         'verbose': 0,
#         'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
#         'depth': trial.suggest_int('depth', 4, 8),
#         'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0),
#         'iterations': 1000,
#         'early_stopping_rounds': 30
#     }

#     cv_results = cv(
#         params=params,
#         pool=train_pool,
#         fold_count=3,
#         shuffle=False,
#         type='TimeSeries',
#         plot=False
#     )

#     # Track both metrics
#     best_mae = cv_results['test-MAE-mean'].min()
#     best_rmse = cv_results['test-RMSE-mean'].min()

#     return best_mae, best_rmse

# # --- 2. Run Multi-Objective Optimization ---
# print("Starting Hyperparameter Tuning (Tracking MAE & RMSE)...")
# study = optuna.create_study(directions=['minimize', 'minimize'])
# study.optimize(objective, n_trials=20)

# # --- 3. Pick the best trial and find optimal iterations ---
# # We pick the trial with the lowest MAE
# best_trial = min(study.best_trials, key=lambda t: t.values[0])

# print(f"\nBest Trial selected (MAE: {best_trial.values[0]:.4f}, RMSE: {best_trial.values[1]:.4f})")

# # Find best iterations for the winning set on 5 folds for stability
# final_params_temp = {
#     'loss_function': 'Huber:delta=20.0',
#     'eval_metric': 'MAE',
#     **best_trial.params
# }

# print("Calculating final optimal iterations...")
# final_cv = cv(params=final_params_temp, pool=train_pool, fold_count=5, shuffle=False, type='TimeSeries', verbose=0)
# best_iteration = int(final_cv['test-MAE-mean'].values.argmin() + 1)

# # --- 4. Save to JSON ---
# best_config = {
#     **best_trial.params,
#     "iterations": best_iteration,
#     "loss_function": 'Huber:delta=20.0',
#     "eval_metric": 'MAE',
#     "final_mae": best_trial.values[0],
#     "final_rmse": best_trial.values[1]
# }

# with open("catboost_rcp_best_params.json", "w") as f:
#     json.dump(best_config, f, indent=4)

# print(f"Tuning complete. Best parameters saved to 'catboost_rcp_best_params.json'")

In [ ]:
# import json
# from catboost import CatBoostRegressor

# # --- 1. Load Parameters ---
# with open("catboost_rcp_best_params.json", "r") as f:
#     best_config = json.load(f)

# # Extract iterations and remove metrics/metadata so CatBoost doesn't crash
# iters = best_config.pop("iterations")
# best_config.pop("final_mae", None)   # Remove if present
# best_config.pop("final_rmse", None)  # Remove if present

# # --- 2. Initialize and Train Model ---
# # Ensure categorical_feature_names and train_pool are defined in your environment
# final_model = CatBoostRegressor(
#     iterations=iters,
#     **best_config,
#     cat_features=categorical_feature_names,
#     random_seed=42,
#     verbose=100
# )

# print("Training final model with saved parameters...")
# final_model.fit(train_pool)

# # --- 3. Save the actual Model weights ---
# final_model.save_model("final_rcp_route_duration_model.cbm")
# print("Model trained and saved as 'final_rcp_route_duration_model.cbm'")

# # --- 4. Predict on Test Set ---
# # Ensure rcp_X_test and drop_cols are defined in your environment
# X_test_features = rcp_X_test.drop(columns=[c for c in drop_cols if c in rcp_X_test.columns])
# predictions = final_model.predict(X_test_features)


# Evaluation metrics

- choose which one to evaluate

In [ ]:
X_test=pkg_X_test.copy()
#X_test=rcp_X_test.copy()

In [ ]:
from sklearn.metrics import mean_absolute_error,root_mean_squared_error
y_test_true = X_test[TARGET_COL]
y_test_pred_subset = predictions

final_rmse = root_mean_squared_error(y_test_true, y_test_pred_subset)
final_mae = mean_absolute_error(y_test_true, y_test_pred_subset)

print(f"\nFinal Model Evaluation on Test Set:")
print(f"RMSE: {final_rmse:.4f} hours")
print(f"MAE: {final_mae:.4f} hours")

In [ ]:
# --- Feature Importance Analysis ---
import pandas as pd
import matplotlib.pyplot as plt

# Get the feature importances from the trained model
feature_importances = final_model.get_feature_importance()
feature_names = X_train_full.columns

# Create a Series for easy sorting and handling
importance_series = pd.Series(feature_importances, index=feature_names)

# Sort the features by importance (descending)
sorted_importance = importance_series.sort_values(ascending=False)

print("\n--- Feature Importances (Top 10) ---")
print(sorted_importance.head(10))

# Optional: Plot the top 10 features
plt.figure(figsize=(10, 6))
sorted_importance.head(10).plot(kind='barh', color='skyblue')
plt.title('Top 10 CatBoost Feature Importances')
plt.xlabel('Importance Score')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
interactions = final_model.get_feature_importance(type='Interaction', prettified=True)
print("\nTop Feature Interactions:")
print(interactions.head(10))

### Analysis of the errors

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Calculate residuals
y_test_target = X_test[TARGET_COL]
residuals = y_test_target - predictions

plt.figure(figsize=(10, 6))
sns.scatterplot(x=predictions, y=residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('Residual Plot: Prediction vs. Error')
plt.xlabel('Predicted Route Duration (Hours)')
plt.ylabel('Residuals (Actual - Predicted)')
plt.show()

In [ ]:
# 1. Calculate the raw error (not absolute)
results = pd.DataFrame({
    'Actual': y_test_target,
    'Predicted': predictions
})
results['Residual'] = results['Predicted'] - results['Actual']

# 2. Filter for large errors (greater than 50 hours)
big_errors = results[np.abs(results['Residual']) > 50]

# 3. Count which is more common
missed_route_duration = big_errors[big_errors['Residual'] < 0].shape[0]
model_hallucinations = big_errors[big_errors['Residual'] > 0].shape[0]

print(f"Total Large Errors (>50h): {len(big_errors)}")
print(f"---")
print(f"Missed route_duration (Under-predicted): {missed_route_duration}")
print(f"Model Hallucinations (Over-predicted): {model_hallucinations}")

# Baseline model

## Preprocessing Pipeline
This section consolidates all the preprocessing steps defined above into a reusable pipeline that can be applied to `X_train_full` and `y_train_full`.

In [ ]:
# 1. Ensure chronological order
pkg_X_train = pkg_X_train.sort_values('date').reset_index(drop=True)
pkg_X_test = pkg_X_test.sort_values('date').reset_index(drop=True)

# --- Define Model Parameters and Features ---
TARGET_COL = 'route_duration'

# List of categorical features (Kept country_service as it was helping!)
cat_features = [
    'etablissement_postal',
    'next_etablissement_postal',
    'day_of_week',
    'service_indicator',
    'origin_destination',
    'EVENT_TYPE_CD',
    'hour',
    'month',
    'is_weekend',
    'first_last_week_day',
    'country_service'
]

# Columns to drop
drop_cols = [TARGET_COL, 'MAILITM_FID', 'RECPTCL_FID', 'date', 'serial_number',
             'flow_type', 'country_code', 'destination_country', 'origin_country']

# Prepare the full 80% training set
X_train_full = pkg_X_train.drop(columns=[c for c in drop_cols if c in pkg_X_train.columns])
y_train_full = pkg_X_train[TARGET_COL]

# Identify categorical features by name
categorical_feature_names = [c for c in cat_features if c in X_train_full.columns]

In [ ]:
# !pip install category_encoders

In [ ]:
# --- Import necessary libraries ---
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.preprocessing import TargetEncoder
from category_encoders import CountEncoder

# --- Define feature groups based on encoding strategy ---
# Low cardinality features: One-Hot Encoding
ONE_HOT_COLS = ['EVENT_TYPE_CD', 'day_of_week','is_weekend', 'first_last_week_day']

# High cardinality features: Target Encoding
HIGH_CARDINALITY_COLS = ['etablissement_postal', 'next_etablissement_postal']

# Medium cardinality features: Count Encoding
MEDIUM_CARDINALITY_COLS = ['origin_country', 'service_indicator','country_service']

# Cyclical features: Sin/Cos Transformation
CYCLICAL_COLS = ['hour', 'month']

# Numerical features that should be scaled
NUMERICAL_COLS = [
    'etab_load_1h',
    'route_load_1h',
    'time_since_first_scan',
    'days_since_last_holiday',
    'days_until_next_holiday',
    'time_since_last_scan'
]

print("Feature groups defined successfully!")

In [ ]:
# --- Define encoding functions ---
def cyclical_transformer(period):
    """Create a cyclical encoding function for a given period."""
    def transform(X):
        X = np.asarray(X).astype(float)
        sin = np.sin(2 * np.pi * X / period)
        cos = np.cos(2 * np.pi * X / period)
        return np.c_[sin, cos]
    return transform

# Hour transformer (24-hour cycle)
hour_transformer = FunctionTransformer(
    cyclical_transformer(24),
    feature_names_out=lambda transformer, names: ["hour_sin", "hour_cos"]
)

# Month transformer (12-month cycle)
month_transformer = FunctionTransformer(
    cyclical_transformer(12),
    feature_names_out=lambda transformer, names: ["month_sin", "month_cos"]
)

print("Cyclical transformers created successfully!")

In [ ]:
# --- Build the preprocessing pipeline function ---
def create_preprocessing_pipeline(
    one_hot_cols=ONE_HOT_COLS,
    high_cardinality_cols=HIGH_CARDINALITY_COLS,
    medium_cardinality_cols=MEDIUM_CARDINALITY_COLS,
    numerical_cols=NUMERICAL_COLS,
    scale_numerical=True
):
    """
    Create a comprehensive preprocessing pipeline for the packages dataset.

    Parameters:
    -----------
    one_hot_cols : list
        Columns for one-hot encoding (low cardinality categorical)
    high_cardinality_cols : list
        Columns for target encoding (high cardinality categorical)
    medium_cardinality_cols : list
        Columns for count encoding (medium cardinality categorical)
    numerical_cols : list
        Numerical columns to scale
    scale_numerical : bool
        Whether to apply StandardScaler to numerical features

    Returns:
    --------
    sklearn.compose.ColumnTransformer
        A preprocessing pipeline
    """

    transformers = [
        # Cyclical encoding for hour
        ('hour_cyclical', hour_transformer, ['hour']),

        # Cyclical encoding for month
        ('month_cyclical', month_transformer, ['month']),

        # One-hot encoding for low cardinality categorical features
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False), one_hot_cols),

        # Target encoding for high cardinality features
        ('target_enc', TargetEncoder(), high_cardinality_cols),

        # Count encoding for medium cardinality features
        ('count_enc', CountEncoder(cols=medium_cardinality_cols), medium_cardinality_cols),
    ]

    # Add numerical scaler if requested
    if scale_numerical and numerical_cols:
        transformers.append(
            ('num_scaler', StandardScaler(), numerical_cols)
        )
    elif numerical_cols:
        transformers.append(
            ('num_passthrough', 'passthrough', numerical_cols)
        )

    preprocessor = ColumnTransformer(
        transformers=transformers,
        remainder='drop',  # Drop any columns not explicitly handled
        verbose_feature_names_out=True
    )

    return preprocessor

print("Pipeline function defined successfully!")

In [ ]:
# --- Helper function to preprocess data ---
def preprocess_data(X, y=None, preprocessor=None, fit=True):
    """
    Apply preprocessing to the data.

    Parameters:
    -----------
    X : pd.DataFrame
        Feature dataframe
    y : pd.Series, optional
        Target variable (required for target encoding during fit)
    preprocessor : ColumnTransformer, optional
        Pre-fitted preprocessor (for transform only)
    fit : bool
        Whether to fit the preprocessor or just transform

    Returns:
    --------
    tuple: (X_processed, preprocessor)
        Processed features and the fitted preprocessor
    """
    if preprocessor is None:
        preprocessor = create_preprocessing_pipeline()

    # Set output to pandas DataFrame
    preprocessor.set_output(transform='pandas')

    if fit:
        if y is None:
            raise ValueError("y is required for fitting (needed for TargetEncoder)")
        X_processed = preprocessor.fit_transform(X, y)
    else:
        X_processed = preprocessor.transform(X)

    return X_processed, preprocessor

print("Preprocessing helper function defined successfully!")

## Apply the pipeline to X_train_full and y_train_full

In [ ]:
# --- Apply the pipeline to X_train_full and y_train_full ---
print("Creating preprocessing pipeline...")

# Filter columns that exist in X_train_full
available_one_hot = [c for c in ONE_HOT_COLS if c in X_train_full.columns]
available_high_card = [c for c in HIGH_CARDINALITY_COLS if c in X_train_full.columns]
available_medium_card = [c for c in MEDIUM_CARDINALITY_COLS if c in X_train_full.columns]
available_numerical = [c for c in NUMERICAL_COLS if c in X_train_full.columns]

print(f"One-hot encoding columns: {available_one_hot}")
print(f"Target encoding columns: {available_high_card}")
print(f"Count encoding columns: {available_medium_card}")
print(f"Numerical columns: {available_numerical}")

# Create the preprocessor with available columns
preprocessor = create_preprocessing_pipeline(
    one_hot_cols=available_one_hot,
    high_cardinality_cols=available_high_card,
    medium_cardinality_cols=available_medium_card,
    numerical_cols=available_numerical,
    scale_numerical=True
)

# Fit and transform training data
X_train_processed, fitted_preprocessor = preprocess_data(
    X_train_full,
    y_train_full,
    preprocessor=preprocessor,
    fit=True
)

print(f"\nOriginal X_train_full shape: {X_train_full.shape}")
print(f"Processed X_train shape: {X_train_processed.shape}")
print(f"\nProcessed feature names:")
print(X_train_processed.columns.tolist())

In [ ]:
# --- Transform test data using the fitted preprocessor ---
# Prepare X_test_full similar to X_train_full
X_test_full = pkg_X_test.drop(columns=[c for c in drop_cols if c in pkg_X_test.columns])
y_test_full = pkg_X_test[TARGET_COL]

# Transform test data (fit=False to avoid data leakage)
X_test_processed, _ = preprocess_data(
    X_test_full,
    preprocessor=fitted_preprocessor,
    fit=False
)

print(f"X_test_full shape: {X_test_full.shape}")
print(f"X_test_processed shape: {X_test_processed.shape}")

# Display sample of processed data
print("\n--- Sample of processed training data ---")
X_train_processed.head()

## KNN Regressor with RandomizedSearchCV
Using the processed data from the preprocessing pipeline to train a KNN Regressor with hyperparameter tuning via RandomizedSearchCV.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, make_scorer
from scipy.stats import randint, uniform
import numpy as np
import warnings
warnings.filterwarnings('ignore')



In [ ]:
param_distributions = {
    'n_neighbors': randint(20, 25),           # Number of neighbors (3 to 50)
}


In [ ]:
# Using TimeSeriesSplit to avoid data leakage in time-series data
tscv = TimeSeriesSplit(n_splits=5)

# Initialize KNN Regressor
knn_base = KNeighborsRegressor(n_jobs=-1)

# Set up RandomizedSearchCV
# Using negative MAE as scoring (sklearn convention: higher is better)
random_search = RandomizedSearchCV(
    estimator=knn_base,
    param_distributions=param_distributions,
    n_iter=5,                    # Number of random combinations to try
    scoring='neg_mean_absolute_error',  # Optimize for MAE
    cv=tscv,                      # TimeSeriesSplit cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1,                    # Use all available cores
    return_train_score=True
)


In [ ]:
# Fit the random search
random_search.fit(X_train_processed, y_train_full)

print("\n" + "="*60)
print("RandomizedSearchCV Complete!")
print("="*60)

In [ ]:
for param, value in random_search.best_params_.items():
    print(f"  {param}: {value}")

print(f"\nBest Cross-Validation MAE: {-random_search.best_score_:.4f} hours")

# Get the best model
best_knn_model = random_search.best_estimator_

In [ ]:
#save best model using joblib
import joblib
filename = 'knn-model.joblib'
joblib.dump(best_knn_model, filename)

## Evaluate Best KNN Model on Test Set

In [ ]:
knn_predictions = best_knn_model.predict(X_test_processed)

# Calculate evaluation metrics
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error

knn_mae = mean_absolute_error(y_test_full, knn_predictions)
knn_rmse = root_mean_squared_error(y_test_full, knn_predictions)

# MAE as percentage of mean
mean_actual = np.mean(y_test_full)
knn_mae_pct = (knn_mae / mean_actual) * 100

print("\n" + "="*60)
print("KNN Regressor - Test Set Evaluation")
print("="*60)
print(f"MAE:  {knn_mae:.4f} hours")
print(f"RMSE: {knn_rmse:.4f} hours")
print(f"\nMean Actual route duration: {mean_actual:.4f} hours")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

knn_residuals = y_test_full - knn_predictions

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot 1: Actual vs Predicted
axes[0].scatter(y_test_full, knn_predictions, alpha=0.3, s=10)
axes[0].plot([y_test_full.min(), y_test_full.max()],
             [y_test_full.min(), y_test_full.max()],
             'r--', lw=2, label='Perfect Prediction')
axes[0].set_xlabel('Actual route_duration (hours)')
axes[0].set_ylabel('Predicted route_duration (hours)')
axes[0].set_title('KNN: Actual vs Predicted')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Residual Distribution
axes[1].hist(knn_residuals, bins=50, edgecolor='black', alpha=0.7)
axes[1].axvline(x=0, color='r', linestyle='--', lw=2)
axes[1].set_xlabel('Residual (Actual - Predicted)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('KNN: Residual Distribution')
axes[1].grid(True, alpha=0.3)

# Plot 3: Residuals vs Predicted
axes[2].scatter(knn_predictions, knn_residuals, alpha=0.3, s=10)
axes[2].axhline(y=0, color='r', linestyle='--', lw=2)
axes[2].set_xlabel('Predicted route_duration (hours)')
axes[2].set_ylabel('Residual (hours)')
axes[2].set_title('KNN: Residuals vs Predicted')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print residual statistics
print("\nResidual Statistics:")
print(f"  Mean Residual: {np.mean(knn_residuals):.4f} hours")
print(f"  Std Residual: {np.std(knn_residuals):.4f} hours")
print(f"  Min Residual: {np.min(knn_residuals):.4f} hours")
print(f"  Max Residual: {np.max(knn_residuals):.4f} hours")

## KNN with Log-Transformed Target
Since the delay has a long right tail, we'll try log-transforming the target to see if it improves KNN performance.

**Why this should help:**
- KNN averages neighbors' values, so extreme outliers dominate predictions
- Log transformation compresses outliers (7000h → ~8.85 in log-space)
- Creates a more symmetric distribution better suited for distance-based averaging

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Original distribution
axes[0].hist(y_train_full, bins=100, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('route_duration (hours)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Original route_duration Distribution')
axes[0].axvline(y_train_full.mean(), color='r', linestyle='--', label=f'Mean: {y_train_full.mean():.2f}')
axes[0].axvline(y_train_full.median(), color='g', linestyle='--', label=f'Median: {y_train_full.median():.2f}')
axes[0].legend()

# Log-transformed distribution (using log1p to handle zeros)
y_train_log = np.log1p(y_train_full)
axes[1].hist(y_train_log, bins=100, edgecolor='black', alpha=0.7, color='orange')
axes[1].set_xlabel('Log(1 + route_duration)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Log-Transformed route_duration Distribution')
axes[1].axvline(y_train_log.mean(), color='r', linestyle='--', label=f'Mean: {y_train_log.mean():.2f}')
axes[1].axvline(y_train_log.median(), color='g', linestyle='--', label=f'Median: {y_train_log.median():.2f}')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"Original - Skewness: {y_train_full.skew():.2f}")
print(f"Log-transformed - Skewness: {y_train_log.skew():.2f}")

In [ ]:
y_train_log = np.log1p(y_train_full)  # log(1 + y) to handle zeros
y_test_log = np.log1p(y_test_full)

print(f"Training target - Original range: [{y_train_full.min():.2f}, {y_train_full.max():.2f}]")
print(f"Training target - Log range: [{y_train_log.min():.2f}, {y_train_log.max():.2f}]")

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from scipy.stats import randint

param_distributions_log = {
    'n_neighbors': randint(9, 30),
}

# Set up TimeSeriesSplit
tscv_log = TimeSeriesSplit(n_splits=5)

# Initialize KNN and RandomizedSearchCV
knn_log = KNeighborsRegressor(n_jobs=-1)

random_search_log = RandomizedSearchCV(
    estimator=knn_log,
    param_distributions=param_distributions_log,
    n_iter=5,
    scoring='neg_mean_absolute_error',
    cv=tscv_log,
    verbose=2,
    random_state=42,
    n_jobs=-1,
    return_train_score=True
)

print("RandomizedSearchCV configured for log-transformed target")

In [ ]:
print("Starting RandomizedSearchCV for KNN with log-transformed target...")
print(f"Training data shape: {X_train_processed.shape}")
print(f"Log target shape: {y_train_log.shape}")
print("This may take a while...\n")

random_search_log.fit(X_train_processed, y_train_log)

print("\n" + "="*60)
print("RandomizedSearchCV Complete!")
print("="*60)

In [ ]:
# --- Display Best Hyperparameters (Log Target) ---
print("Best Hyperparameters for KNN with Log-Transformed Target:")
print("-" * 50)
for param, value in random_search_log.best_params_.items():
    print(f"  {param}: {value}")
print("-" * 50)
print(f"\nBest CV Score (Neg MSE on log scale): {random_search_log.best_score_:.4f}")
print(f"Best CV RMSE (on log scale): {np.sqrt(-random_search_log.best_score_):.4f}")

# Get the best model
best_knn_log = random_search_log.best_estimator_
print(f"\nBest KNN Model (Log Target): {best_knn_log}")

In [ ]:
# --- Evaluate KNN (Log Target) on Test Set ---
# Predict on log scale
y_pred_log = best_knn_log.predict(X_test_processed)

# Inverse transform to get predictions on original scale
y_pred_original = np.expm1(y_pred_log)

# Ensure no negative predictions (can happen due to log transform edge cases)
y_pred_original = np.maximum(y_pred_original, 0)

print("Predictions converted back to original scale using exp(x) - 1")
print(f"Prediction range: [{y_pred_original.min():.2f}, {y_pred_original.max():.2f}] hours")

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Calculate metrics on original scale for fair comparison
knn_log_mae = mean_absolute_error(y_test_full, y_pred_original)
knn_log_rmse = np.sqrt(mean_squared_error(y_test_full, y_pred_original))
knn_log_mape = np.mean(np.abs((y_test_full - y_pred_original) / (y_test_full + 1e-8))) * 100

print("="*60)
print("KNN with Log-Transformed Target - Test Set Performance")
print("(Evaluated on ORIGINAL scale after inverse transform)")
print("="*60)
print(f"  MAE:  {knn_log_mae:.2f} hours")
print(f"  RMSE: {knn_log_rmse:.2f} hours")
print("="*60)

In [ ]:
# --- Visualize Predictions: Original vs Log Target KNN ---
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Get predictions from original KNN for comparison
y_pred_original_knn = best_knn_model.predict(X_test_processed)

# 1. Actual vs Predicted scatter plot
ax1 = axes[0]
ax1.scatter(y_test_full, y_pred_original_knn, alpha=0.3, label='Original Target', s=10)
ax1.scatter(y_test_full, y_pred_original, alpha=0.3, label='Log Target', s=10)
max_val = max(y_test_full.max(), y_pred_original_knn.max(), y_pred_original.max())
ax1.plot([0, max_val], [0, max_val], 'r--', label='Perfect Prediction')
ax1.set_xlabel('Actual route_duration (hours)')
ax1.set_ylabel('Predicted route_duration (hours)')
ax1.set_title('Actual vs Predicted')
ax1.legend()
ax1.set_xlim(0, np.percentile(y_test_full, 99))
ax1.set_ylim(0, np.percentile(y_test_full, 99))

# 2. Residual distribution
ax2 = axes[1]
residuals_orig = y_test_full - y_pred_original_knn
residuals_log = y_test_full - y_pred_original
ax2.hist(residuals_orig, bins=50, alpha=0.5, label=f'Original (std={residuals_orig.std():.1f})', density=True)
ax2.hist(residuals_log, bins=50, alpha=0.5, label=f'Log (std={residuals_log.std():.1f})', density=True)
ax2.axvline(x=0, color='r', linestyle='--')
ax2.set_xlabel('Residual (hours)')
ax2.set_ylabel('Density')
ax2.set_title('Residual Distribution')
ax2.legend()
ax2.set_xlim(-100, 100)

# 3. Prediction distribution comparison
ax3 = axes[2]
ax3.hist(y_pred_original_knn, bins=50, alpha=0.5, label='Original Target KNN', density=True)
ax3.hist(y_pred_original, bins=50, alpha=0.5, label='Log Target KNN', density=True)
ax3.hist(y_test_full, bins=50, alpha=0.3, label='Actual', density=True)
ax3.set_xlabel('route_duration (hours)')
ax3.set_ylabel('Density')
ax3.set_title('Prediction Distribution vs Actual')
ax3.legend()
ax3.set_xlim(0, np.percentile(y_test_full, 99))

plt.tight_layout()
plt.show()

# Anomaly Detection

## Overview

This section implements a comprehensive anomaly detection framework to identify unusual delivery patterns and performance issues. The analysis detects packages with abnormal delivery durations that deviate significantly from predicted baselines.

### Detection Strategy

We implement a **multi-method ensemble approach** combining three complementary techniques:

1. **Statistical Methods (Z-Score & IQR)** - Detect extreme residuals using statistical thresholds
2. **Machine Learning (Isolation Forest)** - Identify unusual patterns across multiple feature dimensions
3. **Ensemble Scoring** - Combine methods for robust anomaly confidence assessment (0-3 scale)

### Expected Outcomes

- Labeled dataset with binary anomaly flags and confidence scores
- Quantification of anomaly prevalence across routes and time periods
- Prioritized list of problematic delivery patterns

## 1. Data Preparation

Prepare the test dataset for anomaly detection by computing prediction residuals and organizing features for analysis.

**Key Operations:**
- Calculate residuals (actual - predicted duration)
- Include temporal and categorical features
- Optimize memory with efficient data types
- Establish baseline statistics

In [ ]:
# Prepare data for anomaly detection
import numpy as np
import pandas as pd
from scipy import stats

# Calculate residuals (prediction errors)
y_actual = X_test[TARGET_COL]
residuals = y_actual - predictions

# Add residuals to test data for analysis (use pkg_X_test to include time features)
anomaly_df = pkg_X_test.copy()
anomaly_df['predicted_duration'] = predictions
anomaly_df['residual'] = residuals
anomaly_df['absolute_residual'] = np.abs(residuals)

# Convert categorical columns to category dtype for memory efficiency
for col in ['etablissement_postal', 'next_etablissement_postal', 'EVENT_TYPE_CD']:
    if col in anomaly_df.columns:
        anomaly_df[col] = anomaly_df[col].astype('category')

print(f"✓ Data prepared for anomaly detection")
print(f"Shape: {anomaly_df.shape}")
print(f"Memory usage: {anomaly_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"\nResidual statistics:")
print(residuals.describe())

### Residual Distribution Visualization

Visualize prediction errors across the test set to understand model performance.

**Key Elements:**
- Zero Error Line: Perfect predictions baseline
- ±2σ Bounds: Contains ~95% of predictions
- Color Intensity: Magnitude of prediction error

In [ ]:
# Visualize Residuals Across Dataset
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(figsize=(16, 6))

sample_indices = np.arange(len(residuals))
ax.scatter(sample_indices, residuals, alpha=0.3, s=5, c=np.abs(residuals),
           cmap='Reds', rasterized=True)
ax.axhline(0, color='black', linestyle='-', linewidth=2, alpha=0.7, label='Zero Error')
ax.axhline(residuals.mean() + 2*residuals.std(), color='black', linestyle='--',
           linewidth=1.5, alpha=0.6, label='±2σ')
ax.axhline(residuals.mean() - 2*residuals.std(), color='black', linestyle='--',
           linewidth=1.5, alpha=0.6)
ax.set_xlabel('Sample Index', fontsize=12, fontweight='bold')
ax.set_ylabel('Residual (hours)', fontsize=12, fontweight='bold')
ax.set_title('Residuals Across Dataset', fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(alpha=0.3, linestyle=':')

plt.tight_layout()
plt.show()

print(f"\n✓ Residual visualization complete!")
print(f"  Mean Absolute Error (MAE): {np.abs(residuals).mean():.2f} hours")
print(f"  Root Mean Squared Error (RMSE): {np.sqrt((residuals**2).mean()):.2f} hours")
print(f"  Percentage within ±24h: {((np.abs(residuals) <= 24).sum() / len(residuals) * 100):.2f}%")

## 2. Z-Score Method

Statistical anomaly detection based on standard deviation from the mean.

**Threshold:** |Z| > 3 (99.7% confidence interval)  
**Formula:** $Z = \frac{(x - \mu)}{\sigma}$

**Advantages:**
- Simple and interpretable
- Objective mathematical thresholds
- Grounded in statistical theory

**Limitations:**
- Assumes normal distribution
- Univariate (only considers residual magnitude)
- Sensitive to overall variance

In [ ]:
# Statistical anomaly detection using Z-Score
# Calculate Z-scores for residuals
z_scores = np.abs(stats.zscore(residuals))

# Flag anomalies: Z-score > 3 (99.7% confidence interval)
z_threshold = 3
anomaly_df['z_score'] = z_scores
anomaly_df['anomaly_zscore'] = (z_scores > z_threshold).astype(int)

print(f"Z-Score Method:")
print(f"Threshold: {z_threshold}")
print(f"Anomalies detected: {anomaly_df['anomaly_zscore'].sum()} ({anomaly_df['anomaly_zscore'].mean()*100:.2f}%)")
print(f"\nZ-score statistics:")
print(f"Mean: {z_scores.mean():.3f}")
print(f"Median: {np.median(z_scores):.3f}")
print(f"Max: {z_scores.max():.3f}")

In [ ]:
# Visualize Z-Score Distribution
fig, ax = plt.subplots(figsize=(12, 6))

ax.hist(anomaly_df[anomaly_df['anomaly_zscore']==0]['z_score'], bins=50,
        alpha=0.6, color='steelblue', label='Normal', edgecolor='black')
ax.hist(anomaly_df[anomaly_df['anomaly_zscore']==1]['z_score'], bins=30,
        alpha=0.9, color='red', label='Anomalies', edgecolor='black')
ax.axvline(z_threshold, color='orange', linestyle='--', linewidth=3,
           label=f'Threshold (Z={z_threshold})', alpha=0.8)

ax.set_xlabel('Z-Score', fontsize=12, fontweight='bold')
ax.set_ylabel('Frequency', fontsize=12, fontweight='bold')
ax.set_title('Z-Score Distribution', fontsize=14, fontweight='bold')
ax.legend(loc='upper right', fontsize=10)
ax.grid(alpha=0.3, linestyle=':')

plt.tight_layout()
plt.show()

zscore_anomalies = anomaly_df[anomaly_df['anomaly_zscore'] == 1]
print(f"✓ Z-Score visualization complete: {len(zscore_anomalies)} anomalies detected ({len(zscore_anomalies)/len(anomaly_df)*100:.2f}%)")

## 3. IQR Method

Quartile-based outlier detection using interquartile range.

**Bounds:** $Q_1 - 1.5 \times IQR$ and $Q_3 + 1.5 \times IQR$  
**Detection:** Values outside bounds are flagged as anomalies

**Advantages:**
- Robust to extreme outliers
- Distribution-free
- Easy to visualize

**Limitations:**
- Fixed threshold multiplier
- Univariate approach
- Treats positive/negative residuals equally

In [ ]:
# IQR (Interquartile Range) Method
Q1 = residuals.quantile(0.25)
Q3 = residuals.quantile(0.75)
IQR = Q3 - Q1

# Define outlier bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Flag anomalies outside IQR bounds
anomaly_df['anomaly_iqr'] = ((residuals < lower_bound) | (residuals > upper_bound)).astype(int)

print(f"\nIQR Method:")
print(f"Q1: {Q1:.2f}, Q3: {Q3:.2f}, IQR: {IQR:.2f}")
print(f"Lower bound: {lower_bound:.2f}, Upper bound: {upper_bound:.2f}")
print(f"Anomalies detected: {anomaly_df['anomaly_iqr'].sum()} ({anomaly_df['anomaly_iqr'].mean()*100:.2f}%)")

In [ ]:
# Visualize IQR Detection Results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

# Left: Scatter plot
iqr_anomalies = anomaly_df[anomaly_df['anomaly_iqr'] == 1]
iqr_normal = anomaly_df[anomaly_df['anomaly_iqr'] == 0]

ax1.scatter(iqr_normal['predicted_duration'], iqr_normal[TARGET_COL],
            alpha=0.2, s=10, color='green', label=f'Normal (n={len(iqr_normal):,})',
            rasterized=True)
ax1.scatter(iqr_anomalies['predicted_duration'], iqr_anomalies[TARGET_COL],
            alpha=0.9, s=80, color='orange', marker='s', linewidths=2,
            label=f'IQR Anomalies (n={len(iqr_anomalies):,})')

max_val = max(anomaly_df[TARGET_COL].max(), anomaly_df['predicted_duration'].max())
ax1.plot([0, max_val], [0, max_val], 'k--', linewidth=2, alpha=0.6, label='Perfect Prediction')

ax1.set_xlabel('Predicted Duration (hours)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Actual Duration (hours)', fontsize=12, fontweight='bold')
ax1.set_title('IQR Method: Anomaly Detection', fontsize=14, fontweight='bold')
ax1.legend(loc='upper left', fontsize=10)
ax1.grid(alpha=0.3, linestyle=':')

# Right: Boxplot showing IQR bounds
data_iqr = [iqr_normal['residual'], iqr_anomalies['residual']]
bp = ax2.boxplot(data_iqr, labels=['Normal', 'IQR Anomalies'], patch_artist=True,
                  widths=0.6, showmeans=True, meanline=True)

# Customize colors
bp['boxes'][0].set_facecolor('green')
bp['boxes'][0].set_alpha(0.5)
bp['boxes'][1].set_facecolor('orange')
bp['boxes'][1].set_alpha(0.7)

# Add IQR bounds as horizontal lines
ax2.axhline(lower_bound, color='red', linestyle='--', linewidth=2,
            label=f'Lower: {lower_bound:.1f}h', alpha=0.7)
ax2.axhline(upper_bound, color='red', linestyle='--', linewidth=2,
            label=f'Upper: {upper_bound:.1f}h', alpha=0.7)
ax2.axhline(0, color='black', linestyle='-', linewidth=1, alpha=0.5)

ax2.set_ylabel('Residual (hours)', fontsize=12, fontweight='bold')
ax2.set_title('IQR Method: Residual Distribution', fontsize=14, fontweight='bold')
ax2.legend(loc='upper right', fontsize=10)
ax2.grid(alpha=0.3, axis='y', linestyle=':')

plt.tight_layout()
plt.show()

print(f"✓ IQR visualization complete: {len(iqr_anomalies):,} anomalies detected ({len(iqr_anomalies)/len(anomaly_df)*100:.2f}%)")

## 4. Isolation Forest Method

Unsupervised machine learning algorithm designed for anomaly detection using random decision trees.

**Core Principle:** Anomalies are easier to isolate from normal points  
**Score:** Based on path length (shorter paths = more anomalous)

**Advantages:**
- Multivariate (considers multiple features)
- Distribution-free
- Detects novel patterns
- Scalable

**Limitations:**
- Less interpretable
- Requires parameter tuning
- More computationally intensive

**Implementation:** Tests contamination values (0.005-0.15) to find best agreement with statistical methods. Uses 100 trees for stable predictions.

In [ ]:
# Isolation Forest for anomaly detection
from sklearn.ensemble import IsolationForest

# Extract time features from date if they don't exist
if 'hour' not in anomaly_df.columns and 'date' in anomaly_df.columns:
    anomaly_df['hour'] = pd.to_datetime(anomaly_df['date']).dt.hour
    anomaly_df['day_of_week'] = pd.to_datetime(anomaly_df['date']).dt.dayofweek
    anomaly_df['month'] = pd.to_datetime(anomaly_df['date']).dt.month

# Select numerical features for Isolation Forest
numerical_features = ['residual', 'absolute_residual']

# Add time features if they exist
if 'hour' in anomaly_df.columns:
    numerical_features.append('hour')
if 'day_of_week' in anomaly_df.columns:
    numerical_features.append('day_of_week')
if 'month' in anomaly_df.columns:
    numerical_features.append('month')

# Add engineered features if they exist
if 'etab_load_1h' in anomaly_df.columns:
    numerical_features.append('etab_load_1h')
if 'route_load_1h' in anomaly_df.columns:
    numerical_features.append('route_load_1h')
if 'time_since_first_scan' in anomaly_df.columns:
    numerical_features.append('time_since_first_scan')

# Prepare feature matrix
X_iso = anomaly_df[numerical_features].fillna(0)

print(f"Features used for Isolation Forest: {numerical_features}")
print(f"Feature matrix shape: {X_iso.shape}")

In [ ]:
# Train Isolation Forest - Find best contamination by agreement with Z-Score and IQR

# Test different contamination values
contamination_values = [0.005, 0.01, 0.02, 0.03, 0.05, 0.07, 0.10, 0.15]
best_contamination = None
best_agreement_score = 0
results_comparison = []

print("Testing contamination values for best agreement with Z-Score and IQR...\n")

for cont in contamination_values:
    # Train Isolation Forest with current contamination
    iso_temp = IsolationForest(
        contamination=cont,
        random_state=42,
        n_estimators=100
    )
    iso_pred_temp = iso_temp.fit_predict(X_iso)
    anomaly_if_temp = (iso_pred_temp == -1).astype(int)

    # Calculate agreement with Z-Score and IQR
    zscore_agreement = ((anomaly_df['anomaly_zscore'] == 1) & (anomaly_if_temp == 1)).sum()
    iqr_agreement = ((anomaly_df['anomaly_iqr'] == 1) & (anomaly_if_temp == 1)).sum()

    # Agreement with either method (union)
    either_agreement = ((anomaly_df['anomaly_zscore'] == 1) | (anomaly_df['anomaly_iqr'] == 1)) & (anomaly_if_temp == 1)
    either_count = either_agreement.sum()

    # All three agree
    all_agree = ((anomaly_df['anomaly_zscore'] == 1) & (anomaly_df['anomaly_iqr'] == 1) & (anomaly_if_temp == 1)).sum()

    # Calculate agreement score: weighted combination
    # Higher weight for all-three agreement, moderate for either agreement
    agreement_score = (all_agree * 3) + (either_count * 1)

    total_if_anomalies = anomaly_if_temp.sum()

    print(f"Contamination: {cont:.3f}")
    print(f"  IF Anomalies: {total_if_anomalies} ({total_if_anomalies/len(anomaly_df)*100:.2f}%)")
    print(f"  Agreement with Z-Score: {zscore_agreement}")
    print(f"  Agreement with IQR: {iqr_agreement}")
    print(f"  Agreement with either: {either_count}")
    print(f"  All three agree: {all_agree}")
    print(f"  Agreement Score: {agreement_score}")
    print()

    results_comparison.append({
        'contamination': cont,
        'if_anomalies': total_if_anomalies,
        'zscore_agreement': zscore_agreement,
        'iqr_agreement': iqr_agreement,
        'either_agreement': either_count,
        'all_three_agree': all_agree,
        'agreement_score': agreement_score
    })

    # Track best contamination
    if agreement_score > best_agreement_score:
        best_agreement_score = agreement_score
        best_contamination = cont

print(f"\n{'='*60}")
print(f"Best contamination value: {best_contamination}")
print(f"Best agreement score: {best_agreement_score}")
print(f"{'='*60}\n")

# Train final model with best contamination
iso_forest = IsolationForest(
    contamination=best_contamination,
    random_state=42,
    n_estimators=100
)

iso_predictions = iso_forest.fit_predict(X_iso)

# Convert to binary (1 for anomaly, 0 for normal)
anomaly_df['anomaly_iforest'] = (iso_predictions == -1).astype(int)

# Get anomaly scores (lower scores = more anomalous)
anomaly_df['iforest_score'] = iso_forest.score_samples(X_iso)

print(f"\nFinal Isolation Forest Results:")
print(f"Contamination parameter: {best_contamination}")
print(f"Anomalies detected: {anomaly_df['anomaly_iforest'].sum()} ({anomaly_df['anomaly_iforest'].mean()*100:.2f}%)")
print(f"\nAnomaly score statistics:")
print(f"Mean: {anomaly_df['iforest_score'].mean():.3f}")
print(f"Min (most anomalous): {anomaly_df['iforest_score'].min():.3f}")
print(f"Max (least anomalous): {anomaly_df['iforest_score'].max():.3f}")

In [ ]:
# Visualize Isolation Forest Detection Results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 6))

# Left: Scatter plot
iforest_anomalies = anomaly_df[anomaly_df['anomaly_iforest'] == 1]
iforest_normal = anomaly_df[anomaly_df['anomaly_iforest'] == 0]

ax1.scatter(iforest_normal['predicted_duration'], iforest_normal[TARGET_COL],
            alpha=0.2, s=10, color='purple', label=f'Normal (n={len(iforest_normal):,})',
            rasterized=True)
ax1.scatter(iforest_anomalies['predicted_duration'], iforest_anomalies[TARGET_COL],
            alpha=0.9, s=80, color='magenta', marker='^', linewidths=2,
            label=f'IF Anomalies (n={len(iforest_anomalies):,})')

max_val = max(anomaly_df[TARGET_COL].max(), anomaly_df['predicted_duration'].max())
ax1.plot([0, max_val], [0, max_val], 'k--', linewidth=2, alpha=0.6, label='Perfect Prediction')

ax1.set_xlabel('Predicted Duration (hours)', fontsize=12, fontweight='bold')
ax1.set_ylabel('Actual Duration (hours)', fontsize=12, fontweight='bold')
ax1.set_title('Isolation Forest Method: Anomaly Detection', fontsize=14, fontweight='bold')
ax1.legend(loc='upper left', fontsize=10)
ax1.grid(alpha=0.3, linestyle=':')

# Right: Anomaly score distribution
ax2.hist(iforest_normal['iforest_score'], bins=50, alpha=0.6, color='purple',
         label='Normal', edgecolor='black')
ax2.hist(iforest_anomalies['iforest_score'], bins=30, alpha=0.9, color='magenta',
         label='Anomalies', edgecolor='black')

# Add threshold line (approximate decision boundary)
threshold = anomaly_df[anomaly_df['anomaly_iforest']==1]['iforest_score'].max()
ax2.axvline(threshold, color='red', linestyle='--', linewidth=3,
            label=f'Decision Boundary ≈ {threshold:.3f}', alpha=0.8)

ax2.set_xlabel('Isolation Forest Anomaly Score', fontsize=12, fontweight='bold')
ax2.set_ylabel('Frequency', fontsize=12, fontweight='bold')
ax2.set_title('Anomaly Score Distribution (lower = more anomalous)', fontsize=14, fontweight='bold')
ax2.legend(loc='upper left', fontsize=10)
ax2.grid(alpha=0.3, linestyle=':')

plt.tight_layout()
plt.show()

print(f"✓ Isolation Forest visualization complete: {len(iforest_anomalies):,} anomalies detected ({len(iforest_anomalies)/len(anomaly_df)*100:.2f}%)")

## 5. Ensemble Method & Comparison

Combine all three methods for robust anomaly detection with confidence scoring.

### Ensemble Score Calculation

$$\text{Anomaly Score} = \text{Z-Score Flag} + \text{IQR Flag} + \text{Isolation Forest Flag}$$

**Score Interpretation:**
- **0**: No methods detected anomaly (normal)
- **1**: One method detected (weak signal)
- **2**: Two methods agree (moderate confidence)
- **3**: All methods agree (high confidence)

**Classification:** `is_anomaly = 1` if score ≥ 2

**Advantages:**
- Reduced false positives through consensus
- Probabilistic confidence scoring
- Combines statistical and ML strengths

In [ ]:
# Ensemble Scoring: Combine all methods
# Calculate ensemble anomaly score (average of all methods)
anomaly_df['anomaly_score'] = (
    anomaly_df['anomaly_zscore'] +
    anomaly_df['anomaly_iqr'] +
    anomaly_df['anomaly_iforest']
) / 3

# High-confidence anomalies: flagged by at least 2 out of 3 methods
anomaly_df['is_anomaly'] = (anomaly_df['anomaly_score'] >= 0.67).astype(int)

print(f"\nEnsemble Anomaly Detection Results:")
print(f"\nMethod comparison:")
print(f"Z-Score anomalies: {anomaly_df['anomaly_zscore'].sum()}")
print(f"IQR anomalies: {anomaly_df['anomaly_iqr'].sum()}")
print(f"Isolation Forest anomalies: {anomaly_df['anomaly_iforest'].sum()}")
print(f"\nEnsemble (≥2 methods agree): {anomaly_df['is_anomaly'].sum()} ({anomaly_df['is_anomaly'].mean()*100:.2f}%)")

# Show distribution of ensemble scores
print(f"\nEnsemble score distribution:")
print(anomaly_df['anomaly_score'].value_counts().sort_index())

In [ ]:
# Comprehensive Method Comparison Visualization
fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Prepare data
methods_data = {
    'Z-Score': anomaly_df['anomaly_zscore'],
    'IQR': anomaly_df['anomaly_iqr'],
    'Isolation Forest': anomaly_df['anomaly_iforest'],
    'Ensemble': anomaly_df['is_anomaly']
}

colors_methods = {'Z-Score': 'red', 'IQR': 'orange', 'Isolation Forest': 'magenta', 'Ensemble': 'crimson'}

# Top row: Actual vs Predicted for each method
for idx, (method, mask_col) in enumerate(methods_data.items()):
    ax = fig.add_subplot(gs[0, idx if idx < 3 else 0])

    method_anomalies = anomaly_df[mask_col == 1]
    method_normal = anomaly_df[mask_col == 0]

    ax.scatter(method_normal['predicted_duration'], method_normal[TARGET_COL],
               alpha=0.15, s=5, color='lightblue', label=f'Normal')
    ax.scatter(method_anomalies['predicted_duration'], method_anomalies[TARGET_COL],
               alpha=0.9, s=50, color=colors_methods[method], marker='x', linewidths=2,
               label=f'Anomalies (n={len(method_anomalies)})')

    max_val = max(anomaly_df[TARGET_COL].max(), anomaly_df['predicted_duration'].max())
    ax.plot([0, max_val], [0, max_val], 'k--', linewidth=1.5, alpha=0.5)

    ax.set_xlabel('Predicted (h)', fontsize=10, fontweight='bold')
    ax.set_ylabel('Actual (h)', fontsize=10, fontweight='bold')
    ax.set_title(f'{method}\n{len(method_anomalies)} anomalies ({len(method_anomalies)/len(anomaly_df)*100:.2f}%)',
                 fontsize=12, fontweight='bold')
    ax.legend(loc='upper left', fontsize=8)
    ax.grid(alpha=0.2, linestyle=':')

# Middle row: Venn diagram comparison and overlap analysis
ax_venn = fig.add_subplot(gs[1, 0])

# Calculate overlaps
zscore_set = set(anomaly_df[anomaly_df['anomaly_zscore']==1].index)
iqr_set = set(anomaly_df[anomaly_df['anomaly_iqr']==1].index)
iforest_set = set(anomaly_df[anomaly_df['anomaly_iforest']==1].index)

overlap_all = zscore_set & iqr_set & iforest_set
overlap_zs_iqr = (zscore_set & iqr_set) - iforest_set
overlap_zs_if = (zscore_set & iforest_set) - iqr_set
overlap_iqr_if = (iqr_set & iforest_set) - zscore_set
only_zs = zscore_set - iqr_set - iforest_set
only_iqr = iqr_set - zscore_set - iforest_set
only_if = iforest_set - zscore_set - iqr_set

overlap_data = {
    'All 3 methods': len(overlap_all),
    'Z-Score & IQR': len(overlap_zs_iqr),
    'Z-Score & IF': len(overlap_zs_if),
    'IQR & IF': len(overlap_iqr_if),
    'Only Z-Score': len(only_zs),
    'Only IQR': len(only_iqr),
    'Only IF': len(only_if)
}

bars = ax_venn.barh(list(overlap_data.keys()), list(overlap_data.values()),
                     color=['darkred', 'red', 'orange', 'yellow', 'lightcoral', 'lightyellow', 'pink'],
                     edgecolor='black', linewidth=1.5)

for i, (bar, val) in enumerate(zip(bars, overlap_data.values())):
    ax_venn.text(val + 10, bar.get_y() + bar.get_height()/2, f'{val}',
                 va='center', fontsize=10, fontweight='bold')

ax_venn.set_xlabel('Number of Anomalies', fontsize=11, fontweight='bold')
ax_venn.set_title('Method Overlap Analysis', fontsize=13, fontweight='bold')
ax_venn.grid(alpha=0.3, axis='x', linestyle=':')

# Middle center: Method agreement matrix
ax_matrix = fig.add_subplot(gs[1, 1])

methods_list = ['Z-Score', 'IQR', 'IF']
agreement_matrix = np.zeros((3, 3))

for i, method1 in enumerate(['anomaly_zscore', 'anomaly_iqr', 'anomaly_iforest']):
    for j, method2 in enumerate(['anomaly_zscore', 'anomaly_iqr', 'anomaly_iforest']):
        if i == j:
            agreement_matrix[i, j] = anomaly_df[method1].sum()
        else:
            agreement_matrix[i, j] = ((anomaly_df[method1] == 1) & (anomaly_df[method2] == 1)).sum()

im = ax_matrix.imshow(agreement_matrix, cmap='Reds', aspect='auto')
ax_matrix.set_xticks(range(3))
ax_matrix.set_yticks(range(3))
ax_matrix.set_xticklabels(methods_list, fontsize=10)
ax_matrix.set_yticklabels(methods_list, fontsize=10)

for i in range(3):
    for j in range(3):
        text = ax_matrix.text(j, i, int(agreement_matrix[i, j]),
                              ha="center", va="center", color="white" if agreement_matrix[i, j] > 500 else "black",
                              fontsize=12, fontweight='bold')

ax_matrix.set_title('Agreement Matrix\n(overlapping detections)', fontsize=12, fontweight='bold')
plt.colorbar(im, ax=ax_matrix, fraction=0.046, pad=0.04)

# Middle right: Ensemble score distribution
ax_ensemble = fig.add_subplot(gs[1, 2])

score_counts = anomaly_df['anomaly_score'].value_counts().sort_index()
score_labels = ['0.0\n(None)', '0.33\n(1/3)', '0.67\n(2/3)', '1.0\n(All)']
colors_score = ['green', 'gold', 'orange', 'crimson']

bars_score = ax_ensemble.bar(range(len(score_counts)), score_counts.values,
                               color=colors_score, edgecolor='black', linewidth=2, alpha=0.8)

for bar, count in zip(bars_score, score_counts.values):
    height = bar.get_height()
    percentage = (count / len(anomaly_df)) * 100
    ax_ensemble.text(bar.get_x() + bar.get_width()/2., height,
                     f'{count:,}\n({percentage:.1f}%)',
                     ha='center', va='bottom', fontsize=9, fontweight='bold')

ax_ensemble.set_xticks(range(len(score_counts)))
ax_ensemble.set_xticklabels(score_labels, fontsize=10)
ax_ensemble.set_ylabel('Count', fontsize=11, fontweight='bold')
ax_ensemble.set_title('Ensemble Score Distribution\n(method agreement)', fontsize=12, fontweight='bold')
ax_ensemble.grid(alpha=0.3, axis='y', linestyle=':')

# Bottom row: Residual distributions for each method
for idx, (method, mask_col) in enumerate(methods_data.items()):
    if idx >= 3:  # Skip ensemble in this row
        break

    ax = fig.add_subplot(gs[2, idx])

    method_anomalies = anomaly_df[mask_col == 1]
    method_normal = anomaly_df[mask_col == 0]

    ax.hist(method_normal['residual'], bins=50, alpha=0.5, color='steelblue',
            label='Normal', edgecolor='black')
    ax.hist(method_anomalies['residual'], bins=30, alpha=0.8, color=colors_methods[method],
            label='Anomalies', edgecolor='black')
    ax.axvline(0, color='black', linestyle='--', linewidth=2, alpha=0.6)

    ax.set_xlabel('Residual (hours)', fontsize=10, fontweight='bold')
    ax.set_ylabel('Frequency', fontsize=10, fontweight='bold')
    ax.set_title(f'{method}: Residual Distribution', fontsize=11, fontweight='bold')
    ax.legend(loc='upper right', fontsize=8)
    ax.grid(alpha=0.3, linestyle=':')

plt.suptitle('Comprehensive Anomaly Detection Method Comparison', fontsize=16, fontweight='bold', y=0.995)
plt.show()

print("COMPREHENSIVE METHOD COMPARISON SUMMARY")
print(f"\nIndividual Method Results:")
print(f"  Z-Score:          {anomaly_df['anomaly_zscore'].sum():,} anomalies ({anomaly_df['anomaly_zscore'].mean()*100:.2f}%)")
print(f"  IQR:              {anomaly_df['anomaly_iqr'].sum():,} anomalies ({anomaly_df['anomaly_iqr'].mean()*100:.2f}%)")
print(f"  Isolation Forest: {anomaly_df['anomaly_iforest'].sum():,} anomalies ({anomaly_df['anomaly_iforest'].mean()*100:.2f}%)")
print(f"\nMethod Overlap:")
print(f"  All 3 agree:      {len(overlap_all)} anomalies")
print(f"  2 methods agree:  {len(overlap_zs_iqr) + len(overlap_zs_if) + len(overlap_iqr_if)} anomalies")
print(f"  Single method:    {len(only_zs) + len(only_iqr) + len(only_if)} anomalies")
print(f"\nFINAL ENSEMBLE RESULT: {anomaly_df['is_anomaly'].sum()} anomalies ({anomaly_df['is_anomaly'].mean()*100:.2f}%)")
print(f"   (Using ≥2 methods agreement threshold)")


## 6. Country & Establishment Analysis

In [ ]:
# Anomaly Characterization Summary
anomalies = anomaly_df[anomaly_df['is_anomaly'] == 1]
normal = anomaly_df[anomaly_df['is_anomaly'] == 0]

print(f"Total Samples: {len(anomaly_df):,}")
print(f"Anomalies: {len(anomalies)} ({len(anomalies)/len(anomaly_df)*100:.2f}%)")
print(f"Normal: {len(normal)} ({len(normal)/len(anomaly_df)*100:.2f}%)")

if len(anomalies) > 0:
    print(f"\nResidual Analysis:")
    print(f"  Anomalies: μ={anomalies['residual'].mean():.1f}h, σ={anomalies['residual'].std():.1f}h")
    print(f"  Normal: μ={normal['residual'].mean():.1f}h, σ={normal['residual'].std():.1f}h")

    print(f"\nAnomaly Types:")
    over_pred = anomalies[anomalies['residual'] < 0]
    under_pred = anomalies[anomalies['residual'] > 0]
    print(f"  Over-predictions (faster): {len(over_pred)} ({len(over_pred)/len(anomalies)*100:.1f}%)")
    print(f"  Under-predictions (slower): {len(under_pred)} ({len(under_pred)/len(anomalies)*100:.1f}%)")

In [ ]:
# Enhanced Boxplot with Statistical Annotations
fig, (ax_box, ax_stats) = plt.subplots(1, 2, figsize=(16, 7), gridspec_kw={'width_ratios': [2, 1]})

# Left: Boxplot
data_to_plot = [normal['residual'], anomalies['residual']]
box = ax_box.boxplot(data_to_plot, labels=['Normal', 'Anomaly'], patch_artist=True,
                     widths=0.6, showmeans=True, meanline=True)

# Customize colors and styling
colors = ['steelblue', 'crimson']
for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.6)
    patch.set_linewidth(2)

for element in ['whiskers', 'fliers', 'means', 'medians', 'caps']:
    plt.setp(box[element], linewidth=2)

ax_box.axhline(0, color='black', linestyle='--', linewidth=2, alpha=0.7, label='Perfect Prediction')
ax_box.set_ylabel('Residual (Actual - Predicted) in Hours', fontsize=13, fontweight='bold')
ax_box.set_title('Residual Distribution Comparison', fontsize=15, fontweight='bold')
ax_box.grid(alpha=0.3, axis='y', linestyle=':')
ax_box.legend(fontsize=10)

# Add individual data points for anomalies (semi-transparent)
ax_box.scatter(np.ones(len(anomalies)) * 2 + np.random.normal(0, 0.04, len(anomalies)),
               anomalies['residual'], alpha=0.3, s=20, color='darkred', zorder=3)

# Right: Statistical Summary Table
ax_stats.axis('tight')
ax_stats.axis('off')

# Calculate comprehensive statistics
stats_data = [
    ['Metric', 'Normal', 'Anomaly', 'Difference'],
    ['Count', f"{len(normal):,}", f"{len(anomalies)}", f"{len(normal)-len(anomalies):,}"],
    ['Mean (μ)', f"{normal['residual'].mean():.2f}h", f"{anomalies['residual'].mean():.2f}h",
     f"{anomalies['residual'].mean() - normal['residual'].mean():.2f}h"],
    ['Median', f"{normal['residual'].median():.2f}h", f"{anomalies['residual'].median():.2f}h",
     f"{anomalies['residual'].median() - normal['residual'].median():.2f}h"],
    ['Std Dev (σ)', f"{normal['residual'].std():.2f}h", f"{anomalies['residual'].std():.2f}h",
     f"{anomalies['residual'].std() - normal['residual'].std():.2f}h"],
    ['Min', f"{normal['residual'].min():.2f}h", f"{anomalies['residual'].min():.2f}h", '-'],
    ['Max', f"{normal['residual'].max():.2f}h", f"{anomalies['residual'].max():.2f}h", '-'],
    ['Q1 (25%)', f"{normal['residual'].quantile(0.25):.2f}h", f"{anomalies['residual'].quantile(0.25):.2f}h", '-'],
    ['Q3 (75%)', f"{normal['residual'].quantile(0.75):.2f}h", f"{anomalies['residual'].quantile(0.75):.2f}h", '-'],
    ['MAE', f"{np.abs(normal['residual']).mean():.2f}h", f"{np.abs(anomalies['residual']).mean():.2f}h",
     f"{np.abs(anomalies['residual']).mean() / np.abs(normal['residual']).mean():.1f}x"]
]

table = ax_stats.table(cellText=stats_data, cellLoc='left', loc='center',
                       colWidths=[0.25, 0.25, 0.25, 0.25])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2.5)

# Style the header row
for i in range(4):
    table[(0, i)].set_text_props(weight='bold', color='white')
    table[(0, i)].set_facecolor('#4472C4')

# Alternate row colors
for i in range(1, len(stats_data)):
    for j in range(4):
        if i % 2 == 0:
            table[(i, j)].set_facecolor('#E7E6E6')
        table[(i, j)].set_linewidth(1.5)
        table[(i, j)].set_edgecolor('black')

ax_stats.set_title('Statistical Summary', fontsize=14, fontweight='bold', pad=20)

plt.suptitle(f'Anomaly Analysis: {len(anomalies)} Anomalies out of {len(anomaly_df):,} Total Samples',
             fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

print(f"\n✓ Statistical comparison complete: Anomalies show {anomalies['residual'].mean() / normal['residual'].mean():.1f}x higher mean residual")

In [ ]:
# Route & Establishment Analysis
if len(anomalies) > 0 and 'etablissement_postal' in anomalies.columns:
    print("\nRoute & Establishment Analysis")
    print("="*50)

    # Top 5 Origin Establishments
    print("\nTop 5 Origin Establishments:")
    etab_counts = anomalies['etablissement_postal'].value_counts().head(5)
    for rank, (etab, count) in enumerate(etab_counts.items(), 1):
        pct = (count / len(anomalies)) * 100
        etab_data = anomalies[anomalies['etablissement_postal'] == etab]
        print(f"  {rank}. {etab}: {count} anomalies ({pct:.1f}%), Avg Error: {etab_data['absolute_residual'].mean():.1f}h")

    # Top 5 Destination Establishments
    if 'next_etablissement_postal' in anomalies.columns:
        print("\nTop 5 Destination Establishments:")
        next_etab_counts = anomalies['next_etablissement_postal'].value_counts().head(5)
        for rank, (etab, count) in enumerate(next_etab_counts.items(), 1):
            pct = (count / len(anomalies)) * 100
            etab_data = anomalies[anomalies['next_etablissement_postal'] == etab]
            print(f"  {rank}. {etab}: {count} anomalies ({pct:.1f}%), Avg Error: {etab_data['absolute_residual'].mean():.1f}h")

        # Top 5 Problematic Routes
        print("\nTop 5 Most Problematic Routes:")
        anomalies['route_pair'] = anomalies['etablissement_postal'].astype(str) + ' → ' + anomalies['next_etablissement_postal'].astype(str)
        route_counts = anomalies['route_pair'].value_counts().head(5)
        for rank, (route, count) in enumerate(route_counts.items(), 1):
            route_data = anomalies[anomalies['route_pair'] == route]
            print(f"  {rank}. {route}")
            print(f"      {count} anomalies ({count/len(anomalies)*100:.1f}%), Avg Error: {route_data['absolute_residual'].mean():.1f}h")

    print("\n" + "="*50)

In [ ]:
# Country Analysis
if len(anomalies) > 0 and 'MAILITM_FID' in anomalies.columns:
    print("\nCountry Analysis")

    def parse_mailitm_fid(id_str):
        if pd.isna(id_str) or str(id_str) == 'nan':
            return None, None, None
        return str(id_str)[0:2], str(id_str)[2:11], str(id_str)[11:14].strip()

    anomalies_country = anomalies.copy()
    anomalies_country[['service_ind', 'serial_num', 'country_code']] = anomalies_country['MAILITM_FID'].apply(
        lambda x: pd.Series(parse_mailitm_fid(x))
    )

    anomaly_df_country = anomaly_df.copy()
    anomaly_df_country[['service_ind', 'serial_num', 'country_code']] = anomaly_df_country['MAILITM_FID'].apply(
        lambda x: pd.Series(parse_mailitm_fid(x))
    )

    country_names = {
        'DZ': 'Algeria', 'FR': 'France', 'US': 'United States', 'AE': 'UAE', 'GB': 'United Kingdom',
        'CA': 'Canada', 'DE': 'Germany', 'IT': 'Italy', 'ES': 'Spain', 'CN': 'China'
    }

    print("\nTop 5 Countries with Most Anomalies:")
    country_anomaly_counts = anomalies_country['country_code'].value_counts().head(5)

    for rank, (country_code, anom_count) in enumerate(country_anomaly_counts.items(), 1):
        country_name = country_names.get(country_code, country_code)
        total = anomaly_df_country[anomaly_df_country['country_code'] == country_code].shape[0]
        rate = (anom_count / total) * 100 if total > 0 else 0
        country_data = anomalies_country[anomalies_country['country_code'] == country_code]
        avg_error = country_data['absolute_residual'].mean()

        print(f"  {rank}. {country_name} ({country_code}): {anom_count} anomalies ({rate:.2f}% rate), Avg Error: {avg_error:.1f}h")



## 7. Save Results

Save anomaly detection results to CSV files in `data/interim/` directory.

### Output Files

**1. packages_anomaly_detection.csv**
- Complete test dataset with all detection scores and flags
- Includes: identifiers, route info, temporal features, predictions, residuals, detection flags, ensemble scores

**2. packages_anomalies_only.csv**
- Filtered dataset containing only flagged anomalies (`is_anomaly = 1`)
- For quick analysis and operational reporting

In [ ]:
# Save anomaly detection results
import os

# Create output directory if it doesn't exist
output_dir = '../data/interim'
os.makedirs(output_dir, exist_ok=True)

# Select columns to save
columns_to_save = [
    'MAILITM_FID', 'RECPTCL_FID', 'date',
    'etablissement_postal', 'next_etablissement_postal',
    'EVENT_TYPE_CD', 'hour', 'day_of_week', 'month',
    TARGET_COL, 'predicted_duration', 'residual', 'absolute_residual',
    'z_score', 'anomaly_zscore', 'anomaly_iqr', 'anomaly_iforest',
    'iforest_score', 'anomaly_score', 'is_anomaly'
]

# Filter to only include columns that exist
columns_to_save = [col for col in columns_to_save if col in anomaly_df.columns]

# Save full results
output_file = os.path.join(output_dir, 'packages_anomaly_detection.csv')
anomaly_df[columns_to_save].to_csv(output_file, index=False)
print(f"✓ Saved full results to: {output_file}")

# Save only anomalies
anomalies_file = os.path.join(output_dir, 'packages_anomalies_only.csv')
anomaly_df[anomaly_df['is_anomaly'] == 1][columns_to_save].to_csv(anomalies_file, index=False)
print(f"✓ Saved anomalies only to: {anomalies_file}")

print(f"\nSummary:")
print(f"Total records: {len(anomaly_df)}")
print(f"Anomalies detected: {anomaly_df['is_anomaly'].sum()} ({anomaly_df['is_anomaly'].mean()*100:.2f}%)")
print(f"Normal records: {(anomaly_df['is_anomaly'] == 0).sum()}")

# Comprehensive Anomaly Insights

Consolidated findings from anomaly detection analysis combining country-level and establishment-level insights for actionable intelligence.

## Report Structure

### 1. Country-Level Analysis (Top 3)
- Volume metrics, anomaly rates, performance impact
- National contribution to total anomalies

### 2. Establishment-Level Breakdown
- **Top 5 Problematic Origins**: Anomaly volume, rate, error magnitude
- **Top 5 Problematic Destinations**: Delivery concentration and error metrics

### 3. Route-Level Intelligence
- **Top 3 Most Problematic Routes**: Anomaly frequency, prediction errors, contribution to country burden

### 4. Detection Method Insights
- Method breakdown by Z-Score, IQR, and Isolation Forest
- Agreement patterns across methods

### 5. Temporal Patterns
- Weekly distribution identifying systematic operational issues

### 6. Overall System Statistics
- Global summary and key performance indicators

## Applications

- **Immediate Action**: Prioritized intervention list
- **Trend Monitoring**: Track patterns over time
- **Capacity Planning**: Identify infrastructure needs
- **Quality Assurance**: Verify delivery SLAs

**Interpretation:**
- Anomaly rates >5% indicate systematic issues
- Error magnitudes >24h suggest infrastructure/staffing problems
- Concentration patterns identify critical intervention points

In [ ]:
# FINAL COMPREHENSIVE SUMMARY: Country Analysis with Establishment Details

if len(anomalies) > 0 and 'MAILITM_FID' in anomalies.columns:
    print("="*100)
    print(" " * 25 + "COMPREHENSIVE ANOMALY SUMMARY BY COUNTRY AND ESTABLISHMENT")
    print("="*100)

    # Get top 3 countries from earlier analysis
    top_countries = anomalies_country['country_code'].value_counts().head(3)

    for rank, (country_code, country_anomaly_count) in enumerate(top_countries.items(), 1):
        country_name = country_names.get(country_code, country_code)
        country_anomalies_detail = anomalies_country[anomalies_country['country_code'] == country_code]
        country_total = anomaly_df_country[anomaly_df_country['country_code'] == country_code]

        # Calculate country-level metrics
        anomaly_rate = (country_anomaly_count / len(country_total)) * 100 if len(country_total) > 0 else 0
        pct_of_all_anomalies = (country_anomaly_count / len(anomalies)) * 100

        avg_residual = country_anomalies_detail['residual'].mean()
        avg_actual_duration = country_anomalies_detail[TARGET_COL].mean()
        avg_predicted_duration = country_anomalies_detail['predicted_duration'].mean()

        print(f"\n{'='*100}")
        print(f"#{rank}. {country_name.upper()} ({country_code})")
        print(f"{'='*100}")

        # Country Overview
        print(f"\nCOUNTRY OVERVIEW:")
        print(f"   • Total Samples: {len(country_total):,}")
        print(f"   • Anomalies: {country_anomaly_count:,} ({anomaly_rate:.2f}% of country samples)")
        print(f"   • Share of All Anomalies: {pct_of_all_anomalies:.1f}%")
        print(f"   • Avg Prediction Error: {avg_residual:.1f} hours")
        print(f"   • Avg Actual Duration: {avg_actual_duration:.1f}h | Avg Predicted: {avg_predicted_duration:.1f}h")

        # Establishment Analysis for this country
        if 'etablissement_postal' in country_anomalies_detail.columns:
            print(f"\nTOP 5 PROBLEMATIC ESTABLISHMENTS (ORIGIN) IN {country_name.upper()}:")
            print(f"   {'Rank':<6} {'Establishment':<20} {'Anomalies':<12} {'% of Country':<15} {'Anomaly Rate':<15} {'Avg Error (h)'}")
            print(f"   {'-'*85}")

            etab_in_country = country_anomalies_detail['etablissement_postal'].value_counts().head(5)

            for etab_rank, (etab_code, etab_anom_count) in enumerate(etab_in_country.items(), 1):
                # Get total samples from this establishment in this country
                etab_total_in_country = country_total[country_total['etablissement_postal'] == etab_code].shape[0]
                etab_anomaly_rate = (etab_anom_count / etab_total_in_country) * 100 if etab_total_in_country > 0 else 0
                pct_of_country_anomalies = (etab_anom_count / country_anomaly_count) * 100

                # Get average error for this establishment
                etab_data = country_anomalies_detail[country_anomalies_detail['etablissement_postal'] == etab_code]
                etab_avg_error = etab_data['absolute_residual'].mean()

                print(f"   {etab_rank:<6} {etab_code:<20} {etab_anom_count:<12} {pct_of_country_anomalies:<14.1f}% "
                      f"{etab_anomaly_rate:<14.1f}% {etab_avg_error:>10.1f}")

        # Destination Establishment Analysis
        if 'next_etablissement_postal' in country_anomalies_detail.columns:
            print(f"\nTOP 5 PROBLEMATIC DESTINATIONS IN {country_name.upper()}:")
            print(f"   {'Rank':<6} {'Destination':<20} {'Anomalies':<12} {'% of Country':<15} {'Avg Error (h)'}")
            print(f"   {'-'*70}")

            dest_in_country = country_anomalies_detail['next_etablissement_postal'].value_counts().head(5)

            for dest_rank, (dest_code, dest_anom_count) in enumerate(dest_in_country.items(), 1):
                pct_of_country_anomalies = (dest_anom_count / country_anomaly_count) * 100

                # Get average error for this destination
                dest_data = country_anomalies_detail[country_anomalies_detail['next_etablissement_postal'] == dest_code]
                dest_avg_error = dest_data['absolute_residual'].mean()

                print(f"   {dest_rank:<6} {dest_code:<20} {dest_anom_count:<12} {pct_of_country_anomalies:<14.1f}% {dest_avg_error:>10.1f}")

        # Top problematic routes within this country
        if 'etablissement_postal' in country_anomalies_detail.columns and 'next_etablissement_postal' in country_anomalies_detail.columns:
            print(f"\nTOP 3 MOST PROBLEMATIC ROUTES IN {country_name.upper()}:")

            country_anomalies_detail_temp = country_anomalies_detail.copy()
            country_anomalies_detail_temp['route_pair'] = (
                country_anomalies_detail_temp['etablissement_postal'].astype(str) + ' → ' +
                country_anomalies_detail_temp['next_etablissement_postal'].astype(str)
            )

            route_counts_country = country_anomalies_detail_temp['route_pair'].value_counts().head(3)

            for route_rank, (route, route_count) in enumerate(route_counts_country.items(), 1):
                route_data = country_anomalies_detail_temp[country_anomalies_detail_temp['route_pair'] == route]
                route_avg_error = route_data['absolute_residual'].mean()
                route_avg_actual = route_data[TARGET_COL].mean()
                route_avg_predicted = route_data['predicted_duration'].mean()

                print(f"\n   #{route_rank}. {route}")
                print(f"      • Anomalies: {route_count} ({route_count/country_anomaly_count*100:.1f}% of country anomalies)")
                print(f"      • Avg Error: {route_avg_error:.1f}h")
                print(f"      • Avg Actual Duration: {route_avg_actual:.1f}h | Predicted: {route_avg_predicted:.1f}h")

        # Detection Method Breakdown
        if 'anomaly_zscore' in country_anomalies_detail.columns:
            zscore_count = country_anomalies_detail['anomaly_zscore'].sum()
            iqr_count = country_anomalies_detail['anomaly_iqr'].sum()
            iforest_count = country_anomalies_detail['anomaly_iforest'].sum()

            print(f"\nDETECTION METHOD BREAKDOWN:")
            print(f"   • Z-Score: {zscore_count} ({zscore_count/country_anomaly_count*100:.1f}%)")
            print(f"   • IQR: {iqr_count} ({iqr_count/country_anomaly_count*100:.1f}%)")
            print(f"   • Isolation Forest: {iforest_count} ({iforest_count/country_anomaly_count*100:.1f}%)")

        # Temporal Patterns
        if 'day_of_week' in country_anomalies_detail.columns:
            day_map = {0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri', 5: 'Sat', 6: 'Sun'}
            day_dist = country_anomalies_detail['day_of_week'].value_counts().sort_index()

            print(f"\nWEEKLY DISTRIBUTION:")
            day_bars = "   "
            for day, count in day_dist.items():
                pct = (count / country_anomaly_count) * 100
                day_name = day_map.get(day, str(day))
                day_bars += f"{day_name}: {count} ({pct:.0f}%)  |  "
            print(day_bars)

    # Overall Summary Statistics
    print(f"\n\n{'='*100}")
    print(f" " * 35 + "OVERALL SUMMARY STATISTICS")
    print(f"{'='*100}\n")

    print(f"DETECTION SUMMARY:")
    print(f"   • Total Test Samples: {len(anomaly_df):,}")
    print(f"   • Total Anomalies Detected: {len(anomalies):,} ({len(anomalies)/len(anomaly_df)*100:.2f}%)")
    print(f"   • Z-Score Anomalies: {anomalies['anomaly_zscore'].sum():,} ({anomalies['anomaly_zscore'].sum()/len(anomalies)*100:.1f}% of anomalies)")
    print(f"   • IQR Anomalies: {anomalies['anomaly_iqr'].sum():,} ({anomalies['anomaly_iqr'].sum()/len(anomalies)*100:.1f}% of anomalies)")
    print(f"   • Isolation Forest Anomalies: {anomalies['anomaly_iforest'].sum():,} ({anomalies['anomaly_iforest'].sum()/len(anomalies)*100:.1f}% of anomalies)")

    print(f"\nKEY INSIGHTS:")
    print(f"   • Top Country: {country_names.get(top_countries.index[0], top_countries.index[0])} with {top_countries.iloc[0]:,} anomalies")
    print(f"   • Average Prediction Error: {anomalies['absolute_residual'].mean():.1f} hours")
    print(f"   • Maximum Prediction Error: {anomalies['absolute_residual'].max():.1f} hours")
    print(f"   • Countries with Anomalies: {anomalies_country['country_code'].nunique()}")
    print(f"   • Establishments with Anomalies: {anomalies['etablissement_postal'].nunique()}")

    print(f"\n{'='*100}")
    print("Analysis complete. Use the detailed breakdowns above to identify and address specific problem areas.")
    print(f"{'='*100}\n")

# Clustering

In [ ]:
clus_packages_df.info()

In [ ]:
clus_receptacles_df.info()

### Null Values Handling

- `next_etablissement_postal` is the only remaining feature having null values
- For each dataset, we'll look at the `next_etablissement_postal`'s values and take the most frequent one corresponding to a specific `etablissement_potsal`'s value

In [ ]:
def get_etab_mapping(df):
    # 1. Calculate the global mode once (to use as a safe fallback)
    global_mode = df['next_etablissement_postal'].mode().iat[0]

    # 2. Get the mode for every group at once
    # This creates a Series where index = etablissement, value = most frequent next
    modes_per_group = df.groupby('etablissement_postal')['next_etablissement_postal'].agg(
        lambda x: x.mode().iat[0] if not x.mode().empty else global_mode
    )

    # 3. Convert to dictionary
    etablissement_dict = modes_per_group.to_dict()
    return etablissement_dict

In [ ]:
pkg_etab_mapping = get_etab_mapping(clus_packages_df)
rcp_etab_mapping = get_etab_mapping(clus_receptacles_df)

In [ ]:
null_mask = clus_packages_df['next_etablissement_postal'].isna()

# Apply to original dataframe directly
clus_packages_df.loc[null_mask, 'next_etablissement_postal'] = (
    clus_packages_df.loc[null_mask, 'etablissement_postal'].map(pkg_etab_mapping)
)

clus_packages_df['next_etablissement_postal'].isna().sum()

* We'll drop the columns that affect clustering's results, and keep only columns that indicate similarity between packages/receptacles.
* Make sure that `year` and `day_of_month` columns exist in both datasets

In [ ]:
clus_packages_df['year'] = clus_packages_df['date'].dt.year
clus_packages_df['day_of_month'] = clus_packages_df['date'].dt.day
clus_packages_df = clus_packages_df.drop(columns=['RECPTCL_FID', 'MAILITM_FID', 'date', 'serial_number'])

In [ ]:
clus_packages_df.columns

In [ ]:
clus_receptacles_df['year'] = clus_receptacles_df['date'].dt.year
clus_receptacles_df['day_of_month'] = clus_receptacles_df['date'].dt.day
clus_receptacles_df = clus_receptacles_df.drop(columns=['RECPTCL_FID', 'date'])

In [ ]:
clus_receptacles_df.columns

In [ ]:
null_mask = clus_receptacles_df['next_etablissement_postal'].isna()

# Apply to original dataframe directly
clus_receptacles_df.loc[null_mask, 'next_etablissement_postal'] = (
    clus_receptacles_df.loc[null_mask, 'etablissement_postal'].map(rcp_etab_mapping)
)

clus_receptacles_df['next_etablissement_postal'].isna().sum()

In [ ]:
# !pip install kmodes

In [ ]:
from kmodes.kmodes import KModes

# Iterating through k values (clusters)
for k in range(5, 30, 5):
    # Fixed n_clusters to k, and set n_init to a reasonable constant like 2
    km_pkg = KModes(n_clusters=k, init='Huang', n_init=2, verbose=0)
    km_rcp = KModes(n_clusters=k, init='Huang', n_init=2, verbose=0)

    # Fit and predict
    clusters_pkg = km_pkg.fit_predict(clus_packages_df)
    clusters_rcp = km_rcp.fit_predict(clus_receptacles_df)

    # Print evaluation (Cost)
    print(f"K={k} | Packages Cost: {km_pkg.cost_:.2f} | Receptacles Cost: {km_rcp.cost_:.2f}")

#### Clustering on Raw Data

* We Only keep columns that existed in the original dataset
* We decompose the `date` columns into multiple features to allow fair and less-restrictive comparison (not on minutes or seconds)

In [ ]:
raw_clustering_pkg = clus_packages_df[['etablissement_postal', 'next_etablissement_postal', 'EVENT_TYPE_CD', 'day_of_month', 'month', 'year', 'hour', 'day_of_week']]
raw_clustering_rcp = clus_receptacles_df[['etablissement_postal', 'next_etablissement_postal', 'EVENT_TYPE_CD', 'day_of_month', 'month', 'year', 'hour', 'day_of_week']]

In [ ]:
# Iterating through k values (clusters)
for k in range(5, 30, 5):
    # Fixed n_clusters to k, and set n_init to a reasonable constant like 2
    km_pkg = KModes(n_clusters=k, init='Huang', n_init=2, verbose=0)
    km_rcp = KModes(n_clusters=k, init='Huang', n_init=2, verbose=0)

    # Fit and predict
    raw_clusters_pkg = km_pkg.fit_predict(raw_clustering_pkg)
    raw_clusters_rcp = km_rcp.fit_predict(raw_clustering_rcp)

    # Print evaluation (Cost)
    print(f"K={k} | Packages Cost: {km_pkg.cost_:.2f} | Receptacles Cost: {km_rcp.cost_:.2f}")